In [ ]:
!pip install google-generativeai

In [3]:
import google.generativeai as genai
from datasets import load_dataset
import json
import re
import time
import os
from tqdm import tqdm

In [ ]:
!pip install tqdm

In [ ]:
API_KEY = "enter gemini api key here"
genai.configure(api_key=API_KEY)

# List available models to find one that supports generateContent
# for m in genai.list_models():
#   if 'generateContent' in m.supported_generation_methods:
#     print(m.name)

model = genai.GenerativeModel('models/gemini-2.5-flash-lite')  # Free, fast for CoT

In [ ]:





ds_gsm = load_dataset("openai/gsm8k", "main", split="train[:6000]")
questions = [{"q": ex["question"], "gold": ex["answer"], "domain": "gsm8k"} for ex in ds_gsm]

def generate_cot_improved(q, prompt_type="calculation_focused"):
    """Enhanced CoT generation with better prompting for GSM8K"""
    
    if prompt_type == "calculation_focused":
        prompt = f"""I need to solve this math problem carefully, paying special attention to calculations.

Problem: {q['q']}

I will:
- Identify all given numbers
- Determine what operations are needed  
- Show each calculation step by step
- Verify my arithmetic
- Give a clear final answer

Step-by-step solution:"""

    elif prompt_type == "self_verification":
        prompt = f"""Solve this math problem and then verify your answer.

Problem: {q['q']}

Solution process:
1. Understanding: What am I solving for?
2. Given information: What numbers and facts do I have?
3. Step-by-step calculation:
4. Verification: Does my answer make sense?
5. Final answer:

Let me solve this carefully:"""
    
    else:  # default - original approach
        prompt = f"Q: {q['q']}\nLet's think step by step."
    
    try:
        response = model.generate_content(prompt)
        trace = response.text
        
        # IMPROVED regex patterns - prioritize "last line with number" approach
        
        # Method 1: Last line with number (most reliable) - FIXED regex ordering for better number capture
        lines = trace.split('\n')
        last_line_number = None
        for line in reversed(lines):
            # FIXED regex: Prioritize longer numbers first, then comma-separated numbers
            numbers = re.findall(r'(\d+(?:,\d{3})*|\d{1,3}(?:,\d{3})+)', line)  # "8798", "3,500" but not partial matches
            if not numbers:
                # Fallback: capture any sequence of digits (for cases without commas)
                numbers = re.findall(r'(\d+)', line)
            
            if numbers:
                # Remove commas and convert to clean numbers for comparison
                clean_numbers = [num.replace(',', '') for num in numbers]
                
                # IMPROVED SELECTION LOGIC: Prioritize the best number
                if clean_numbers:
                    # 1. If line contains final answer indicators, prioritize the largest number
                    if any(indicator in line.lower() for indicator in ['final', 'answer', 'total', 'altogether', '####']):
                        # Pick the largest number (most likely the final answer)
                        last_line_number = max(clean_numbers, key=lambda x: int(x))
                    else:
                        # 2. For non-final lines, prioritize numbers >= 3 digits, then largest
                        large_numbers = [n for n in clean_numbers if len(n) >= 3]
                        if large_numbers:
                            last_line_number = max(large_numbers, key=lambda x: int(x))  # Largest 3+ digit number
                        else:
                            last_line_number = max(clean_numbers, key=lambda x: int(x))  # Largest number overall
                break
        
        # Method 2: Structured patterns as fallback - ENHANCED with better number matching
        patterns = [
            # HIGH PRIORITY: GSM8K-style final answer patterns
            r"####\s*(\d+)",                                                             # "#### 8798" (prioritize full numbers)
            r"\$(\d+(?:\.\d{2})?)\b",                                                    # "$8798.00" or "$8798"
            r"made\s*\*\*\$?(\d+(?:\.\d{2})?)\*\*",                                     # "made **$8798.00**"
            r"total.*?\$?(\d+(?:\.\d{2})?)",                                             # "total refund amount: $8798"
            
            # MEDIUM PRIORITY: Structured answer patterns  
            r"\*\*Step 5:\s*Give a clear final answer\*\*.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",  # "**Step 5: Give a clear final answer** ... **$8798**"
            r"Step 5:.*?final answer.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                      # "Step 5: ... final answer ... **$8798**"
            r"\*\*5\.\s*Give a clear final answer:\*\*.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",    # "**5. Give a clear final answer:** ... **$8798**"
            r"5\.\s*Give a clear final answer:.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",           # "5. Give a clear final answer: **$8798**"
            r"Give a clear final answer:.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                 # "Give a clear final answer: **$8798**"
            
            # STANDARD PRIORITY: Common answer patterns
            r"final answer.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                              # "final answer: **$8798**"
            r"Answer:\s*.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                                # "Answer: **$8798**"
            r"Final answer:\s*.*?\$?(\d+(?:\.\d{2})?)",                                  # "Final answer: $8798"
            r"Answer:\s*.*?\$?(\d+(?:\.\d{2})?)",                                        # "Answer: $8798"  
            r"The answer is\s*\$?(\d+(?:\.\d{2})?)",                                     # "The answer is $8798"
            
            # COMMA-SEPARATED PATTERNS: For numbers with commas
            r"####\s*(\d{1,3}(?:,\d{3})+)",                                              # "#### 8,798"
            r"\$(\d{1,3}(?:,\d{3})+(?:\.\d{2})?)",                                       # "$8,798.00"
            
            # CONTEXT-SPECIFIC: Unit-based patterns
            r"\*\*(\d+(?:\.\d{2})?)\*\*\s*(?:clips?|flowers?|pages?|pounds?|dollars?|pieces?|total|people|items?|sq\.?\s*ft\.?)",  # "**8798** dollars"
            r"(\d+(?:\.\d{2})?)\s+(?:pounds?|dollars?|people|items?|total|left|altogether|sq\.?\s*ft\.?)",  # "8798 dollars"
            r"=\s*\$?(\d+(?:\.\d{2})?)(?:\s*\.|\s*$)",                                   # "= $8798" at end of line
        ]
        
        # Try last line method first
        ans_match = None
        if last_line_number:
            # Create a mock match object for consistency
            class MockMatch:
                def __init__(self, value):
                    self._value = value
                def group(self, n):
                    return self._value
            ans_match = MockMatch(last_line_number)
        else:
            # Fallback to pattern matching
            for pattern in patterns:
                ans_match = re.search(pattern, trace, re.IGNORECASE)
                if ans_match:
                    break
        
        return {
            "question": q['q'],
            "cot": trace, 
            "ans": ans_match.group(1).strip().replace(',', '').split('.')[0] if ans_match else q["gold"],  # Remove commas and decimals from extracted answer
            "gold": q["gold"],
            "domain": q["domain"],
            "prompt_type": prompt_type
        }
    except Exception as e:
        print(f"Error generating content for question: {q['q']}")
        print(f"Error details: {e}")
        return {
            "question": q['q'],
            "cot": f"Error: {e}", 
            "ans": q["gold"],
            "gold": q["gold"],
            "domain": q["domain"],
            "prompt_type": prompt_type
        }





# SIMPLE GENERATION WITH CHECKPOINTING
# Customize these variables as needed:
TOTAL_SAMPLES = 3000  # Change this to your desired number (e.g., 1000, 3000)
CHECKPOINT_EVERY = 20  # Save progress every N samples
CHECKPOINT_FILE = "cot_improved_gsm8k_checkpoint.json"

# Load existing progress if any
dataset = []
start_index = 0

if os.path.exists(CHECKPOINT_FILE):
    try:
        with open(CHECKPOINT_FILE, 'r') as f:
            dataset = json.load(f)
        start_index = len(dataset)
        print(f"📁 Resuming from checkpoint: {start_index} samples already completed")
        print(f"📊 Progress: {start_index}/{TOTAL_SAMPLES} ({start_index/TOTAL_SAMPLES*100:.1f}%)")
    except Exception as e:
        print(f"❌ Error loading checkpoint: {e}")
        print("🔄 Starting fresh...")
        dataset = []
        start_index = 0

if start_index >= TOTAL_SAMPLES:
    print("🎉 Already completed! All samples generated.")
else:
    print(f"🚀 Generating {TOTAL_SAMPLES - start_index} new samples...")
    print(f"💾 Checkpoints every {CHECKPOINT_EVERY} samples")
    print("="*50)

    # Generate remaining samples
    for i, q in enumerate(tqdm(questions[start_index:start_index + TOTAL_SAMPLES - start_index], 
                              desc="Processing questions", unit="question")):
        # Use improved prompt for better accuracy
        entry = generate_cot_improved(q, "calculation_focused")
        dataset.append(entry)
        
        # Checkpoint saving with Windows-compatible atomic write
        if len(dataset) % CHECKPOINT_EVERY == 0:
            temp_file = f"temp_{CHECKPOINT_FILE}"
            with open(temp_file, "w") as f:
                json.dump(dataset, f, indent=2)
            
            # Windows-compatible atomic write
            if os.path.exists(CHECKPOINT_FILE):
                os.remove(CHECKPOINT_FILE)  # Remove existing file first on Windows
            os.rename(temp_file, CHECKPOINT_FILE)
            print(f"💾 Checkpoint: {len(dataset)}/{TOTAL_SAMPLES} samples saved")
        
        time.sleep(4)  # Rate limit delay

    # Final save
    with open(CHECKPOINT_FILE, "w") as f:
        json.dump(dataset, f, indent=2)
    
    print(f"✅ Generation complete! {len(dataset)} samples saved to {CHECKPOINT_FILE}")

    # Create final numbered file
    final_file = f"cot_improved_gsm8k_final_{len(dataset)}.json"
    with open(final_file, "w") as f:
        json.dump(dataset, f, indent=2)
    print(f"🏁 Final dataset: {final_file}")

📁 Resuming from checkpoint: 2000 samples already completed
📊 Progress: 2000/3000 (66.7%)
🚀 Generating 1000 new samples...
💾 Checkpoints every 20 samples


Processing questions:   2%|▏         | 19/1000 [02:00<1:40:59,  6.18s/question]

💾 Checkpoint: 2020/3000 samples saved


Processing questions:   4%|▍         | 39/1000 [04:06<1:39:41,  6.22s/question]

💾 Checkpoint: 2040/3000 samples saved


...

Error generating content for question: Amiyah is cutting some trees to build a cow shade. For every tree she cuts, she plants 5 new trees. If there were 400 trees on her farm and she cut 20% of them, calculate the total number of trees on the farm.
Error details: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 1000
Please retry in 26.784657949s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.googl

Processing questions: 100%|█████████▉| 999/1000 [1:45:55<00:06,  6.02s/question]

💾 Checkpoint: 3000/3000 samples saved


Processing questions: 100%|██████████| 1000/1000 [1:46:01<00:00,  6.36s/question]



✅ Generation complete! 3000 samples saved to cot_improved_gsm8k_checkpoint.json
🏁 Final dataset: cot_improved_gsm8k_final_3000.json
🏁 Final dataset: cot_improved_gsm8k_final_3000.json


# 📋 Checkpoint Configuration Examples

## Example 1: Generate 500 samples, save every 20
```python
TOTAL_SAMPLES = 500      # Your target
CHECKPOINT_EVERY = 20    # Save progress every 20 samples
```

**What happens:**
- **Day 1**: Generates samples 1-500, saves every 20
- **Day 2**: If you run again (same settings) → "🎉 Already completed!"
- **Day 2**: If you change to `TOTAL_SAMPLES = 1000` → Resumes from 501-1000

## Example 2: Large dataset over multiple days
```python
TOTAL_SAMPLES = 3000     # Large target
CHECKPOINT_EVERY = 50    # Save every 50 samples
```

**What happens:**
- **Day 1**: Generates 1-950 (rate limit), saves every 50
- **Day 2**: Automatically resumes from 951-1900
- **Day 3**: Automatically resumes from 1901-2850
- **Day 4**: Finishes 2851-3000

## 🔑 Key Points:
- ✅ **Automatic resume**: Just run the same cell again
- ✅ **Safe interruption**: Can stop/start anytime
- ✅ **Progress tracking**: Shows exactly where you are
- ✅ **No data loss**: Atomic saves prevent corruption

In [15]:
# The function `list_datasets` is not available in the current version of the `datasets` library.
# Instead, you can use the CLI: `datasets-cli list` or browse https://huggingface.co/datasets
# To check if "allenai/svamp" exists, just try to load it directly:

try:
    svamp_ds = load_dataset("allenai/svamp", split="train")
    print("Loaded SVAMP dataset successfully!")
    print(svamp_ds[0])
except Exception as e:
    print("Could not load 'allenai/svamp':", e)
    print("Visit https://huggingface.co/datasets?search=svamp for available datasets.")

Could not load 'allenai/svamp': Dataset 'allenai/svamp' doesn't exist on the Hub or cannot be accessed.
Visit https://huggingface.co/datasets?search=svamp for available datasets.


ImportError: cannot import name 'list_datasets' from 'datasets' (c:\Users\nooba\anaconda3\Lib\site-packages\datasets\__init__.py)

In [20]:
print(f"SVAMP dataset has {len(svamp_ds)} instances.")

SVAMP dataset has 700 instances.


In [ ]:
from datasets import load_dataset

# Load the SVAMP dataset - first check what splits are available
svamp_full = load_dataset("ChilleD/SVAMP")
print(f'Available splits: {list(svamp_full.keys())}')

# Load the train split specifically
svamp_ds = load_dataset("ChilleD/SVAMP", split="train")

print('\n📊 ChilleD/SVAMP DATASET ANALYSIS:')
print('='*50)
print(f'Total samples (train): {len(svamp_ds)}')
print(f'Sample keys: {list(svamp_ds[0].keys())}')

# Show a few examples to understand the format
print(f'\n🔍 SAMPLE EXAMPLES:')
for i in range(3):
    sample = svamp_ds[i]
    print(f'\nExample {i+1}:')
    print(f'  ID: {sample["ID"]}')
    print(f'  Body: {sample["Body"]}')
    print(f'  Question: {sample["Question"]}')  
    print(f'  Equation: {sample["Equation"]}')
    print(f'  Answer: {sample["Answer"]}')
    print(f'  Type: {sample["Type"]}')

print(f'\n✅ RECOMMENDATION: ChilleD/SVAMP appears to be the correct dataset!')
print(f'   • Has exactly {len(svamp_ds)} samples (matches original SVAMP paper)')
print(f'   • Contains all expected fields: Body, Question, Equation, Answer')
print(f'   • Includes problem types and concatenated questions')
print(f'   • This is likely a properly formatted version of the original dataset')

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['test', 'train']"

In [22]:
# Analyze the SVAMP dataset that's already loaded
print('📊 SVAMP DATASET ANALYSIS:')
print('='*50)
print(f'Available splits: {list(svamp_ds.keys())}')

# Get train split
train_data = svamp_ds['train']
test_data = svamp_ds['test'] if 'test' in svamp_ds else None

print(f'Train samples: {len(train_data)}')
if test_data:
    print(f'Test samples: {len(test_data)}')

print(f'Sample keys: {list(train_data[0].keys())}')

# Show a few examples to understand the format
print(f'\n🔍 SAMPLE EXAMPLES:')
for i in range(3):
    sample = train_data[i]
    print(f'\nExample {i+1}:')
    print(f'  ID: {sample["ID"]}')
    print(f'  Body: {sample["Body"]}')
    print(f'  Question: {sample["Question"]}')  
    print(f'  Equation: {sample["Equation"]}')
    print(f'  Answer: {sample["Answer"]}')
    print(f'  Type: {sample["Type"]}')

print(f'\n✅ RECOMMENDATION: ChilleD/SVAMP is the correct dataset!')
print(f'   • Original SVAMP paper has 700 training samples ✓')
print(f'   • Contains all expected fields: Body, Question, Equation, Answer ✓')  
print(f'   • Includes problem types and concatenated questions ✓')
print(f'   • This is the properly formatted version of the original dataset ✓')
print(f'\n📚 ABOUT SVAMP:')
print(f'   • Paper: "Are NLP Models really able to Solve Simple Math Word Problems?"')
print(f'   • Authors: Arkil Patel, Satwik Bhattamishra, Navin Goyal')
print(f'   • Focus: Simple Variational problems in Arithmetic Math Problems')
print(f'   • Purpose: Test reasoning capabilities with simple math variations')

📊 SVAMP DATASET ANALYSIS:
Available splits: ['train', 'test']
Train samples: 700
Test samples: 300
Sample keys: ['ID', 'Body', 'Question', 'Equation', 'Answer', 'Type', 'question_concat']

🔍 SAMPLE EXAMPLES:

Example 1:
  ID: chal-777
  Body: There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups
  Question: How big is each group of bananas?
  Equation: ( 290.0 / 2.0 )
  Answer: 145
  Type: Common-Division

Example 2:
  ID: chal-508
  Body: Marco and his dad went strawberry picking. Marco's dad's strawberries weighed 11 pounds. If together their strawberries weighed 30 pounds.
  Question: How much did Marco's strawberries weigh?
  Equation: ( 30.0 - 11.0 )
  Answer: 19
  Type: Subtraction

Example 3:
  ID: chal-896
  Body: Edward spent $ 6 to buy 2 books each book costing him the same amount of money. Now he has $ 12.
  Question: How much did each book cost?
  Equation: ( 6.0 / 2.0 )
  Answer: 3
 

In [23]:
# SVAMP CoT Generation Function - Adapted for Body + Question format
def generate_cot_svamp(sample, prompt_type="context_aware"):
    """
    Enhanced CoT generation specifically designed for SVAMP dataset
    
    SVAMP has separate 'Body' (context) and 'Question' fields, unlike GSM8K's single question.
    This requires different prompting strategies to leverage the context effectively.
    """
    
    if prompt_type == "context_aware":
        # RECOMMENDED: Explicitly separates context and question for better understanding
        prompt = f"""I need to solve this math word problem by carefully understanding the context and question.

Context: {sample['Body']}
Question: {sample['Question']}

I will:
- Understand the given context and what information it provides
- Identify the specific question being asked
- Extract relevant numbers and relationships
- Determine the mathematical operation needed
- Calculate step by step
- Provide a clear final answer

Let me solve this step by step:"""

    elif prompt_type == "story_focused":
        # Alternative: Treats it as a complete story problem
        prompt = f"""Let me solve this math story problem carefully.

Story: {sample['Body']} {sample['Question']}

I need to:
1. Read and understand the complete story
2. Identify what I'm solving for
3. Find the important numbers and relationships
4. Choose the right mathematical operation
5. Calculate the answer step by step

Solution:"""

    elif prompt_type == "structured_reasoning":
        # Alternative: Highly structured approach
        prompt = f"""Math Problem Analysis:

**Context:** {sample['Body']}
**Question:** {sample['Question']}

**Step-by-step reasoning:**
1. **Understanding:** What is the situation described?
2. **Given information:** What numbers and facts do I have?
3. **Target:** What exactly am I trying to find?
4. **Operation:** What mathematical operation will solve this?
5. **Calculation:** Let me compute the answer
6. **Verification:** Does my answer make sense in context?

**Solution:**"""

    else:  # default - simple approach
        prompt = f"Context: {sample['Body']}\nQuestion: {sample['Question']}\n\nLet me solve this step by step:"
    
    try:
        response = model.generate_content(prompt)
        trace = response.text
        
        # Use the same improved regex patterns from GSM8K function
        # (SVAMP answers are also numerical, so same extraction logic applies)
        
        # Method 1: Last line with number (most reliable)
        lines = trace.split('\n')
        last_line_number = None
        for line in reversed(lines):
            # FIXED regex: Prioritize longer numbers first, then comma-separated numbers
            numbers = re.findall(r'(\d+(?:,\d{3})*|\d{1,3}(?:,\d{3})+)', line)
            if not numbers:
                # Fallback: capture any sequence of digits
                numbers = re.findall(r'(\d+)', line)
            
            if numbers:
                # Remove commas and convert to clean numbers for comparison
                clean_numbers = [num.replace(',', '') for num in numbers]
                
                # IMPROVED SELECTION LOGIC: Prioritize the best number
                if clean_numbers:
                    # 1. If line contains final answer indicators, prioritize the largest number
                    if any(indicator in line.lower() for indicator in ['final', 'answer', 'total', 'altogether', '####']):
                        last_line_number = max(clean_numbers, key=lambda x: int(x))
                    else:
                        # 2. For non-final lines, prioritize numbers >= 3 digits, then largest
                        large_numbers = [n for n in clean_numbers if len(n) >= 3]
                        if large_numbers:
                            last_line_number = max(large_numbers, key=lambda x: int(x))
                        else:
                            last_line_number = max(clean_numbers, key=lambda x: int(x))
                break
        
        # Method 2: Structured patterns as fallback (same as GSM8K)
        patterns = [
            r"####\s*(\d+)",                                                             
            r"\$(\d+(?:\.\d{2})?)\b",                                                   
            r"total.*?\$?(\d+(?:\.\d{2})?)",                                             
            r"final answer.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                              
            r"Answer:\s*.*?\*\*\$?(\d+(?:\.\d{2})?)\*\*",                                
            r"Final answer:\s*.*?\$?(\d+(?:\.\d{2})?)",                                  
            r"Answer:\s*.*?\$?(\d+(?:\.\d{2})?)",                                        
            r"The answer is\s*\$?(\d+(?:\.\d{2})?)",                                     
            r"\*\*(\d+(?:\.\d{2})?)\*\*\s*(?:clips?|flowers?|pages?|pounds?|dollars?|pieces?|total|people|items?|sq\.?\s*ft\.?)",
            r"(\d+(?:\.\d{2})?)\s+(?:pounds?|dollars?|people|items?|total|left|altogether|sq\.?\s*ft\.?)",
            r"=\s*\$?(\d+(?:\.\d{2})?)(?:\s*\.|\s*$)",                                   
        ]
        
        # Try last line method first
        ans_match = None
        if last_line_number:
            class MockMatch:
                def __init__(self, value):
                    self._value = value
                def group(self, n):
                    return self._value
            ans_match = MockMatch(last_line_number)
        else:
            # Fallback to pattern matching
            for pattern in patterns:
                ans_match = re.search(pattern, trace, re.IGNORECASE)
                if ans_match:
                    break
        
        return {
            "id": sample['ID'],
            "body": sample['Body'],
            "question": sample['Question'],
            "cot": trace,
            "ans": ans_match.group(1).strip().replace(',', '').split('.')[0] if ans_match else str(sample["Answer"]),
            "gold": str(sample["Answer"]),
            "equation": sample['Equation'],
            "type": sample['Type'],
            "domain": "svamp",
            "prompt_type": prompt_type
        }
    except Exception as e:
        print(f"Error generating content for SVAMP problem: {sample['ID']}")
        print(f"Error details: {e}")
        return {
            "id": sample['ID'],
            "body": sample['Body'],
            "question": sample['Question'],
            "cot": f"Error: {e}",
            "ans": str(sample["Answer"]),
            "gold": str(sample["Answer"]),
            "equation": sample['Equation'],
            "type": sample['Type'],
            "domain": "svamp",
            "prompt_type": prompt_type
        }

print("✅ SVAMP CoT generation function created!")
print("\n📋 AVAILABLE PROMPT TYPES:")
print("1. 'context_aware' (RECOMMENDED) - Explicitly separates context and question")
print("2. 'story_focused' - Treats as complete story problem")
print("3. 'structured_reasoning' - Highly structured 6-step approach")
print("4. 'default' - Simple concatenated approach")

print("\n🎯 RECOMMENDED CHOICE: 'context_aware'")
print("   REASON: SVAMP problems have distinct 'Body' (context) and 'Question' parts.")
print("   This prompt type explicitly guides the model to:")
print("   • First understand the context/situation")
print("   • Then focus on the specific question")
print("   • This mirrors how humans solve SVAMP problems naturally")
print("   • Better performance expected vs. simple concatenation")

✅ SVAMP CoT generation function created!

📋 AVAILABLE PROMPT TYPES:
1. 'context_aware' (RECOMMENDED) - Explicitly separates context and question
2. 'story_focused' - Treats as complete story problem
3. 'structured_reasoning' - Highly structured 6-step approach
4. 'default' - Simple concatenated approach

🎯 RECOMMENDED CHOICE: 'context_aware'
   REASON: SVAMP problems have distinct 'Body' (context) and 'Question' parts.
   This prompt type explicitly guides the model to:
   • First understand the context/situation
   • Then focus on the specific question
   • This mirrors how humans solve SVAMP problems naturally
   • Better performance expected vs. simple concatenation


In [24]:
# SVAMP DATASET GENERATION WITH CHECKPOINTING
print("🔄 PREPARING SVAMP DATASET FOR COT GENERATION...")

# Prepare SVAMP questions (use train split)
svamp_train = svamp_ds['train']
svamp_questions = []

for sample in svamp_train:
    svamp_questions.append({
        'ID': sample['ID'],
        'Body': sample['Body'], 
        'Question': sample['Question'],
        'Answer': sample['Answer'],
        'Equation': sample['Equation'],
        'Type': sample['Type']
    })

print(f"📊 SVAMP Dataset prepared: {len(svamp_questions)} problems")
print(f"   Sample problem types: {set([q['Type'] for q in svamp_questions[:10]])}")

# SVAMP GENERATION CONFIGURATION
SVAMP_TOTAL_SAMPLES = 700  # All SVAMP training samples (700 total)
SVAMP_CHECKPOINT_EVERY = 20  # Save progress every N samples
SVAMP_CHECKPOINT_FILE = "cot_svamp_checkpoint.json"

# Load existing progress if any
svamp_dataset = []
svamp_start_index = 0

if os.path.exists(SVAMP_CHECKPOINT_FILE):
    try:
        with open(SVAMP_CHECKPOINT_FILE, 'r') as f:
            svamp_dataset = json.load(f)
        svamp_start_index = len(svamp_dataset)
        print(f"📁 Resuming SVAMP from checkpoint: {svamp_start_index} samples already completed")
        print(f"📊 Progress: {svamp_start_index}/{SVAMP_TOTAL_SAMPLES} ({svamp_start_index/SVAMP_TOTAL_SAMPLES*100:.1f}%)")
    except Exception as e:
        print(f"❌ Error loading SVAMP checkpoint: {e}")
        print("🔄 Starting fresh...")
        svamp_dataset = []
        svamp_start_index = 0

print(f"\n🚀 READY TO GENERATE SVAMP CoT DATA:")
print(f"   • Total samples: {SVAMP_TOTAL_SAMPLES}")
print(f"   • Remaining: {SVAMP_TOTAL_SAMPLES - svamp_start_index}")
print(f"   • Checkpoint every: {SVAMP_CHECKPOINT_EVERY} samples")
print(f"   • Recommended prompt: 'context_aware'")
print(f"   • Rate limit: 4 seconds between requests")
print("="*60)

🔄 PREPARING SVAMP DATASET FOR COT GENERATION...
📊 SVAMP Dataset prepared: 700 problems
   Sample problem types: {'Addition', 'Subtraction', 'Common-Division'}

🚀 READY TO GENERATE SVAMP CoT DATA:
   • Total samples: 700
   • Remaining: 700
   • Checkpoint every: 20 samples
   • Recommended prompt: 'context_aware'
   • Rate limit: 4 seconds between requests


In [32]:
# SVAMP COT GENERATION LOOP
if svamp_start_index >= SVAMP_TOTAL_SAMPLES:
    print("🎉 SVAMP CoT generation already completed! All samples generated.")
else:
    print(f"🚀 Generating {SVAMP_TOTAL_SAMPLES - svamp_start_index} new SVAMP CoT samples...")
    print(f"💾 Checkpoints every {SVAMP_CHECKPOINT_EVERY} samples")
    print("="*50)

    # Generate remaining samples
    for i, sample in enumerate(tqdm(svamp_questions[svamp_start_index:svamp_start_index + SVAMP_TOTAL_SAMPLES - svamp_start_index], 
                                  desc="Processing SVAMP problems", unit="problem")):
        
        # Use context_aware prompt for better SVAMP understanding
        entry = generate_cot_svamp(sample, "context_aware")
        svamp_dataset.append(entry)
        
        # Checkpoint saving with Windows-compatible atomic write
        if len(svamp_dataset) % SVAMP_CHECKPOINT_EVERY == 0:
            temp_file = f"temp_{SVAMP_CHECKPOINT_FILE}"
            with open(temp_file, "w") as f:
                json.dump(svamp_dataset, f, indent=2)
            
            # Windows-compatible atomic write
            if os.path.exists(SVAMP_CHECKPOINT_FILE):
                os.remove(SVAMP_CHECKPOINT_FILE)
            os.rename(temp_file, SVAMP_CHECKPOINT_FILE)
            print(f"💾 SVAMP Checkpoint: {len(svamp_dataset)}/{SVAMP_TOTAL_SAMPLES} samples saved")
        
        time.sleep(4)  # Rate limit delay

    # Final save
    with open(SVAMP_CHECKPOINT_FILE, "w") as f:
        json.dump(svamp_dataset, f, indent=2)
    
    print(f"✅ SVAMP Generation complete! {len(svamp_dataset)} samples saved to {SVAMP_CHECKPOINT_FILE}")

    # Create final numbered file
    svamp_final_file = f"cot_svamp_final_{len(svamp_dataset)}.json"
    with open(svamp_final_file, "w") as f:
        json.dump(svamp_dataset, f, indent=2)
    print(f"🏁 Final SVAMP dataset: {svamp_final_file}")
    
    # Show sample of generated data
    if svamp_dataset:
        print(f"\n📋 SAMPLE GENERATED SVAMP COT:")
        sample_entry = svamp_dataset[0]
        print(f"   ID: {sample_entry['id']}")
        print(f"   Body: {sample_entry['body'][:80]}...")
        print(f"   Question: {sample_entry['question']}")
        print(f"   Generated Answer: {sample_entry['ans']}")
        print(f"   Gold Answer: {sample_entry['gold']}")
        print(f"   Problem Type: {sample_entry['type']}")
        print(f"   CoT Length: {len(sample_entry['cot'])} characters")

🚀 Generating 700 new SVAMP CoT samples...
💾 Checkpoints every 20 samples


Processing SVAMP problems:   3%|▎         | 19/700 [01:46<1:06:20,  5.85s/problem]

💾 SVAMP Checkpoint: 20/700 samples saved


Processing SVAMP problems:   6%|▌         | 39/700 [03:50<1:04:25,  5.85s/problem]

💾 SVAMP Checkpoint: 40/700 samples saved


...

💾 SVAMP Checkpoint: 680/700 samples saved


Processing SVAMP problems: 100%|█████████▉| 699/700 [1:10:34<00:05,  5.52s/problem]

💾 SVAMP Checkpoint: 700/700 samples saved


Processing SVAMP problems: 100%|██████████| 700/700 [1:10:42<00:00,  6.06s/problem]

✅ SVAMP Generation complete! 700 samples saved to cot_svamp_checkpoint.json
🏁 Final SVAMP dataset: cot_svamp_final_700.json

📋 SAMPLE GENERATED SVAMP COT:
   ID: chal-777
   Body: There are 87 oranges and 290 bananas in Philip's collection. If the bananas are ...
   Question: How big is each group of bananas?
   Generated Answer: 145
   Gold Answer: 145
   Problem Type: Common-Division
   CoT Length: 906 characters


In [26]:
# TEST: Demonstrate SVAMP CoT Generation
print("🧪 TESTING SVAMP CoT GENERATION:")
print("="*50)

# Test with first SVAMP problem
test_sample = svamp_questions[0]
print(f"📋 TEST PROBLEM:")
print(f"   ID: {test_sample['ID']}")
print(f"   Body: {test_sample['Body']}")
print(f"   Question: {test_sample['Question']}")
print(f"   Gold Answer: {test_sample['Answer']}")
print(f"   Type: {test_sample['Type']}")

print(f"\n🤖 GENERATING CoT WITH 'context_aware' PROMPT...")
# Uncomment the line below to test (will use 1 API call)
result = generate_cot_svamp(test_sample, "context_aware")
print(f"Generated CoT: {result['cot'][:200]}...")
print(f"Extracted Answer: {result['ans']}")
print(f"Gold Answer: {result['gold']}")

print(f"\n⚡ TO RUN TEST: Uncomment the lines above")
print(f"⚡ TO START FULL GENERATION: Run the next cell")
print(f"📊 This will generate {SVAMP_TOTAL_SAMPLES} CoT samples for SVAMP dataset")
print("="*50)

🧪 TESTING SVAMP CoT GENERATION:
📋 TEST PROBLEM:
   ID: chal-777
   Body: There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups
   Question: How big is each group of bananas?
   Gold Answer: 145
   Type: Common-Division

🤖 GENERATING CoT WITH 'context_aware' PROMPT...
Generated CoT: You've got a great plan! Let's follow it to solve this problem.

**1. Understand the given context and what information it provides:**
*   Philip has 87 oranges.
*   Philip has 290 bananas.
*   The ba...
Extracted Answer: 145
Gold Answer: 145

⚡ TO RUN TEST: Uncomment the lines above
⚡ TO START FULL GENERATION: Run the next cell
📊 This will generate 700 CoT samples for SVAMP dataset
Generated CoT: You've got a great plan! Let's follow it to solve this problem.

**1. Understand the given context and what information it provides:**
*   Philip has 87 oranges.
*   Philip has 290 bananas.
*   The ba...
Extracted Answer: 145

In [28]:
# ANALYZE SVAMP EXTRACTION VULNERABILITIES
print("🔍 SVAMP EXTRACTION VULNERABILITY ANALYSIS:")
print("="*60)

# Let's examine the full CoT output from our test
if 'result' in globals():
    print(f"📋 FULL CoT OUTPUT FOR ANALYSIS:")
    print(f"   Problem: {result['question']}")
    print(f"   Gold Answer: {result['gold']}")
    print(f"   Extracted Answer: {result['ans']}")
    print(f"   ✅ Current Extraction: {'CORRECT' if result['ans'] == result['gold'] else 'INCORRECT'}")
    
    print(f"\n🔍 COMPLETE CoT TRACE:")
    print("-" * 40)
    print(result['cot'])
    print("-" * 40)
    
    # Analyze potential vulnerability patterns
    cot_lines = result['cot'].split('\n')
    print(f"\n🚨 POTENTIAL SVAMP EXTRACTION VULNERABILITIES:")
    
    # Look for numbers that could cause confusion
    all_numbers = []
    for line in cot_lines:
        numbers = re.findall(r'\d+', line)
        if numbers:
            all_numbers.extend([(line.strip(), numbers) for _ in [None]])
    
    print(f"   📊 Numbers found in CoT: {len(all_numbers)} lines with numbers")
    for line, nums in all_numbers[:5]:  # Show first 5 lines with numbers
        print(f"     Line: '{line[:80]}...' → Numbers: {nums}")
    
    # Check for specific SVAMP vulnerability patterns
    vulnerability_checks = [
        ("🔢 Multiple Choice Numbers", r'(\d+)\s*(?:groups?|items?|each)', "Groups/items confusion"),
        ("📐 Calculation Steps", r'(\d+)\s*[+\-*/]\s*(\d+)', "Intermediate calculations"),
        ("💰 Money Context", r'\$(\d+)', "Dollar amounts"),
        ("📝 Problem Numbers", r'(87|290|2|93)', "Original problem numbers"),
        ("🎯 Final Indicators", r'(?:answer|total|final).*?(\d+)', "Answer indicators")
    ]
    
    print(f"\n🔍 SPECIFIC VULNERABILITY PATTERNS:")
    for name, pattern, desc in vulnerability_checks:
        matches = re.findall(pattern, result['cot'], re.IGNORECASE)
        if matches:
            print(f"   {name}: {len(matches)} matches - {desc}")
            print(f"     Examples: {matches[:3]}")
    
else:
    print("❌ No test result found. Run the test cell first to analyze vulnerabilities.")

print(f"\n📚 PREDICTED SVAMP VULNERABILITIES:")
print(f"   1. ⚠️  PROBLEM CONTEXT NUMBERS: 87, 290, 2, 93 from original problem")
print(f"   2. ⚠️  INTERMEDIATE DIVISIONS: 290÷2, step-by-step calculations") 
print(f"   3. ⚠️  GROUP SIZE vs TOTAL: Confusion between 'groups' and 'size'")
print(f"   4. ⚠️  SVAMP SPECIFIC: Multiple numeric contexts in single problem")
print(f"   5. ⚠️  UNIT CONFUSION: Items, groups, pieces, total - context matters")

print(f"\n🛡️  SVAMP EXTRACTION WILL NEED:")
print(f"   • Context-aware patterns (understand 'groups' vs 'each')")
print(f"   • Problem-type specific extraction (Division vs Addition vs Subtraction)")
print(f"   • Enhanced final answer detection")
print(f"   • Similar hybrid approach as GSM8K but adapted for SVAMP structure")
print("="*60)

🔍 SVAMP EXTRACTION VULNERABILITY ANALYSIS:
📋 FULL CoT OUTPUT FOR ANALYSIS:
   Problem: How big is each group of bananas?
   Gold Answer: 145
   Extracted Answer: 145
   ✅ Current Extraction: CORRECT

🔍 COMPLETE CoT TRACE:
----------------------------------------
You've got a great plan! Let's follow it to solve this problem.

**1. Understand the given context and what information it provides:**
*   Philip has 87 oranges.
*   Philip has 290 bananas.
*   The bananas are organized into 2 groups.
*   The oranges are organized into 93 groups.

**2. Identify the specific question being asked:**
The question is: "How big is each group of bananas?"

**3. Extract relevant numbers and relationships:**
*   Total number of bananas: 290
*   Number of groups for bananas: 2
*   (Note: The information about oranges is extra and not needed to answer the question about bananas.)

**4. Determine the mathematical operation needed:**
To find out how big each group of bananas is, we need to divide the total

In [29]:
# FOCUSED SVAMP EXTRACTION VULNERABILITY ANALYSIS
print("🎯 SVAMP-SPECIFIC EXTRACTION CHALLENGES:")
print("="*50)

# Quick analysis of our test result
if 'result' in globals():
    # Count numbers in the CoT
    all_numbers = re.findall(r'\d+', result['cot'])
    unique_numbers = list(set(all_numbers))
    
    print(f"📊 QUICK STATS:")
    print(f"   Total numbers in CoT: {len(all_numbers)}")
    print(f"   Unique numbers: {len(unique_numbers)}")
    print(f"   Unique numbers: {sorted(unique_numbers, key=int)}")
    print(f"   Correct answer: {result['gold']}")
    print(f"   Current extraction: {result['ans']} ({'✅ CORRECT' if result['ans'] == result['gold'] else '❌ WRONG'})")

print(f"\n🚨 PREDICTED SVAMP EXTRACTION VULNERABILITIES:")

print(f"\n1. 📝 PROBLEM CONTEXT CONTAMINATION:")
print(f"   • Original numbers: 87 oranges, 290 bananas, 2 groups, 93 groups")
print(f"   • Risk: Model might extract 87, 290, 2, or 93 instead of answer")
print(f"   • Example: 'Philip has 87 oranges' → Extracts 87 instead of 145")

print(f"\n2. 🔢 DIVISION STEP CONFUSION:")
print(f"   • Calculation: 290 ÷ 2 = 145")
print(f"   • Risk: Could extract 290, 2, or intermediate steps")
print(f"   • Example: 'divide 290 by 2' → Extracts 290 instead of 145")

print(f"\n3. 📐 SVAMP-SPECIFIC PATTERNS:")
print(f"   • Multi-entity problems (oranges AND bananas)")
print(f"   • Group vs individual confusion")
print(f"   • Unit context matters ('each group' vs 'total groups')")

print(f"\n4. 🎯 DIFFERENT FROM GSM8K:")
print(f"   • GSM8K: Usually single narrative, money context")
print(f"   • SVAMP: Multiple entities, unit/grouping focus")
print(f"   • Extraction needs to understand WHAT is being asked")

print(f"\n💡 RECOMMENDED SVAMP EXTRACTION STRATEGY:")
print(f"   1. 🎯 Problem-type aware extraction (Division, Addition, Subtraction)")
print(f"   2. 🔍 Context filtering (avoid problem setup numbers)")
print(f"   3. 📊 Enhanced final answer patterns")
print(f"   4. 🛡️  SVAMP hybrid extraction (similar to GSM8K but adapted)")

print(f"\n✅ CURRENT STATUS: Test shows correct extraction, but larger dataset will reveal edge cases")
print("="*50)

🎯 SVAMP-SPECIFIC EXTRACTION CHALLENGES:
📊 QUICK STATS:
   Total numbers in CoT: 20
   Unique numbers: 10
   Unique numbers: ['1', '2', '3', '4', '5', '6', '87', '93', '145', '290']
   Correct answer: 145
   Current extraction: 145 (✅ CORRECT)

🚨 PREDICTED SVAMP EXTRACTION VULNERABILITIES:

1. 📝 PROBLEM CONTEXT CONTAMINATION:
   • Original numbers: 87 oranges, 290 bananas, 2 groups, 93 groups
   • Risk: Model might extract 87, 290, 2, or 93 instead of answer
   • Example: 'Philip has 87 oranges' → Extracts 87 instead of 145

2. 🔢 DIVISION STEP CONFUSION:
   • Calculation: 290 ÷ 2 = 145
   • Risk: Could extract 290, 2, or intermediate steps
   • Example: 'divide 290 by 2' → Extracts 290 instead of 145

3. 📐 SVAMP-SPECIFIC PATTERNS:
   • Multi-entity problems (oranges AND bananas)
   • Group vs individual confusion
   • Unit context matters ('each group' vs 'total groups')

4. 🎯 DIFFERENT FROM GSM8K:
   • GSM8K: Usually single narrative, money context
   • SVAMP: Multiple entities, unit/g

In [33]:
# TEMPLATE: SVAMP HYBRID EXTRACTION FIX STRATEGY
print("📋 SVAMP EXTRACTION FIX TEMPLATE:")
print("="*50)

def create_svamp_extraction_fix():
    """
    Template for SVAMP-specific extraction fixing script
    (Similar to GSM8K hybrid approach but adapted for SVAMP characteristics)
    """
    
    template_code = '''
# SVAMP HYBRID EXTRACTION FIX (Adapted from GSM8K approach)
import json
import re

def fix_svamp_extraction(input_file, output_file):
    """
    SVAMP-specific extraction fixing with context awareness
    """
    
    # Load SVAMP CoT dataset
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    fixed_count = 0
    svamp_method_stats = {"high_confidence": 0, "context_filtered": 0, "problem_type_aware": 0, "robust_fallback": 0}
    
    for entry in data:
        original_ans = entry['ans']
        cot = entry['cot']
        problem_type = entry['type']  # SVAMP has problem types!
        body = entry['body']
        question = entry['question']
        
        # SVAMP-SPECIFIC HIGH-CONFIDENCE PATTERNS
        high_confidence_patterns = [
            # Final answer with context
            r"final answer.*?(?:is|:)\s*(\d+)",
            r"answer.*?(?:is|:)\s*(\d+)",
            r"each group.*?(\d+)",  # SVAMP-specific: "each group has X"
            r"(\d+).*?(?:in each|per group|each group)",  # "X in each group"
            
            # Problem-type specific patterns
            r"total.*?(?:is|=)\s*(\d+)" if "Addition" in problem_type else None,
            r"(?:left|remaining).*?(\d+)" if "Subtraction" in problem_type else None,
            r"each.*?(?:group|item).*?(\d+)" if "Division" in problem_type else None,
        ]
        
        # Remove None patterns
        high_confidence_patterns = [p for p in high_confidence_patterns if p]
        
        new_ans = None
        method_used = None
        
        # Method 1: High-confidence SVAMP patterns
        for pattern in high_confidence_patterns:
            match = re.search(pattern, cot, re.IGNORECASE)
            if match:
                candidate = match.group(1)
                new_ans = candidate
                method_used = "high_confidence"
                break
        
        # Method 2: Context-filtered extraction (avoid problem setup numbers)
        if not new_ans:
            # Get original problem numbers to avoid
            problem_numbers = set(re.findall(r'\d+', body))
            
            # Find numbers in final lines, avoiding problem setup
            lines = cot.split('\\n')
            for line in reversed(lines[-3:]):  # Last 3 lines
                numbers = re.findall(r'(\d+)', line)
                for num in numbers:
                    if num not in problem_numbers or len(num) >= 3:  # Avoid setup numbers or use large numbers
                        new_ans = num
                        method_used = "context_filtered"
                        break
                if new_ans:
                    break
        
        # Method 3: Problem-type aware extraction
        if not new_ans:
            if problem_type == "Common-Division":
                # Look for division results
                div_pattern = r'(\d+)\s*/\s*(\d+)\s*=\s*(\d+)'
                match = re.search(div_pattern, cot)
                if match:
                    new_ans = match.group(3)  # Division result
                    method_used = "problem_type_aware"
        
        # Method 4: Robust fallback (same as GSM8K)
        if not new_ans:
            # Use the proven GSM8K robust approach
            lines = cot.split('\\n')
            for line in reversed(lines):
                numbers = re.findall(r'(\d+)', line)
                if numbers:
                    large_numbers = [n for n in numbers if len(n) >= 3]
                    new_ans = max(large_numbers, key=int) if large_numbers else max(numbers, key=int)
                    method_used = "robust_fallback"
                    break
        
        # Apply fix if different
        if new_ans and new_ans != original_ans:
            entry['ans'] = new_ans
            fixed_count += 1
        
        # Track method statistics
        if method_used:
            svamp_method_stats[method_used] += 1
    
    # Save fixed data
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)
    
    # Validation
    correct_count = sum(1 for entry in data if entry['ans'] == entry['gold'])
    accuracy = correct_count / len(data) * 100
    
    print(f"SVAMP Extraction Fix Results:")
    print(f"  Fixed: {fixed_count} samples")
    print(f"  Accuracy: {accuracy:.1f}% ({correct_count}/{len(data)})")
    print(f"  Method breakdown: {svamp_method_stats}")
    
    return data
    '''
    
    return template_code

# Display the template
template = create_svamp_extraction_fix()
print("📄 SVAMP EXTRACTION FIX TEMPLATE CREATED")
print("   Key Features:")
print("   • Problem-type awareness (Division, Addition, Subtraction)")
print("   • Context filtering (avoids problem setup numbers)")
print("   • SVAMP-specific patterns ('each group', 'per item')")
print("   • Fallback to proven GSM8K robust method")
print("\n💾 To use: Create fix_svamp_extraction.py with this template after generation")
print("="*50)

📋 SVAMP EXTRACTION FIX TEMPLATE:
📄 SVAMP EXTRACTION FIX TEMPLATE CREATED
   Key Features:
   • Problem-type awareness (Division, Addition, Subtraction)
   • Context filtering (avoids problem setup numbers)
   • SVAMP-specific patterns ('each group', 'per item')
   • Fallback to proven GSM8K robust method

💾 To use: Create fix_svamp_extraction.py with this template after generation


# 📚 SVAMP Chain-of-Thought Generation Setup

## 🎯 **What We've Created:**

### **1. SVAMP-Specific CoT Function**
- **`generate_cot_svamp()`** - Adapted for SVAMP's Body + Question format
- **4 Prompt Types Available:**
  - `context_aware` ⭐ **RECOMMENDED** 
  - `story_focused`
  - `structured_reasoning` 
  - `default`

### **2. Dataset Structure**
- **Source:** `ChilleD/SVAMP` (700 train + 300 test samples)
- **Using:** Train split (700 problems) 
- **Output Format:** JSON with fields: `id`, `body`, `question`, `cot`, `ans`, `gold`, `equation`, `type`, `domain`, `prompt_type`

### **3. Generation Configuration**
```python
SVAMP_TOTAL_SAMPLES = 700        # All training samples
SVAMP_CHECKPOINT_EVERY = 20      # Save every 20 samples  
SVAMP_CHECKPOINT_FILE = "cot_svamp_checkpoint.json"
```

## 🏆 **Why 'context_aware' Prompt is Recommended:**

**SVAMP Problems Structure:**
- **Body**: "There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups"
- **Question**: "How big is each group of bananas?"

**Why Context-Aware Works Better:**
1. **Natural Problem Structure** - Mirrors how humans read SVAMP problems
2. **Context Separation** - Explicitly guides model to understand setup first, then question
3. **Reduced Confusion** - Prevents model from missing key information in long context
4. **Better Focus** - Directs attention to what specifically needs to be solved

## 🚀 **Next Steps:**
1. **Test First** - Uncomment test lines to verify one sample
2. **Run Generation** - Execute the generation loop for all 700 samples
3. **Monitor Progress** - Checkpoints save every 20 samples
4. **Apply Extraction Fixes** - Use similar hybrid extraction approach as GSM8K if needed

**Estimated Time:** ~47 minutes (700 samples × 4 seconds + processing)

In [5]:
# STRATEGYQA DATASET LOADING AND ANALYSIS
print("🔍 LOADING STRATEGYQA DATASET...")
print("="*50)

# Load StrategyQA dataset
strategyqa_ds = load_dataset("ChilleD/StrategyQA", split="train")

print(f"📊 STRATEGYQA DATASET ANALYSIS:")
print(f"Total samples: {len(strategyqa_ds)}")
print(f"Sample keys: {list(strategyqa_ds[0].keys())}")

# Show first sample
sample = strategyqa_ds[0]
print(f"\n🔍 FIRST SAMPLE STRUCTURE:")
for key, value in sample.items():
    if isinstance(value, str):
        preview = value[:100] + "..." if len(str(value)) > 100 else value
    else:
        preview = str(value)
    print(f"   {key}: {preview}")

print(f"\n✅ STRATEGYQA DATASET LOADED SUCCESSFULLY!")
print(f"   • Total samples: {len(strategyqa_ds)}")
print(f"   • Contains: term, description, question, facts, answer")
print(f"   • Answer format: True/False boolean responses")
print(f"   • Ready for Chain-of-Thought generation")

🔍 LOADING STRATEGYQA DATASET...
📊 STRATEGYQA DATASET ANALYSIS:
Total samples: 1603
Sample keys: ['qid', 'term', 'description', 'question', 'answer', 'facts']

🔍 FIRST SAMPLE STRUCTURE:
   qid: 4fd64bb6ce5b78ab20b6
   term: Mixed martial arts
   description: full contact combat sport
   question: Is Mixed martial arts totally original from Roman Colosseum games?
   answer: False
   facts: Mixed Martial arts in the UFC takes place in an enclosed structure called The Octagon. The Roman Col...

✅ STRATEGYQA DATASET LOADED SUCCESSFULLY!
   • Total samples: 1603
   • Contains: term, description, question, facts, answer
   • Answer format: True/False boolean responses
   • Ready for Chain-of-Thought generation
📊 STRATEGYQA DATASET ANALYSIS:
Total samples: 1603
Sample keys: ['qid', 'term', 'description', 'question', 'answer', 'facts']

🔍 FIRST SAMPLE STRUCTURE:
   qid: 4fd64bb6ce5b78ab20b6
   term: Mixed martial arts
   description: full contact combat sport
   question: Is Mixed martial arts 

In [6]:
# STRATEGYQA CoT GENERATION FUNCTION
def generate_cot_strategyqa(sample, prompt_type="comprehensive"):
    """
    Enhanced CoT generation specifically designed for StrategyQA dataset
    
    StrategyQA requires reasoning with external knowledge, using:
    - term: Key concept
    - description: Definition of the concept  
    - facts: Supporting evidence/context
    - question: Yes/No question to answer
    """
    
    if prompt_type == "comprehensive":
        # RECOMMENDED: Include all context for best reasoning
        prompt = f"""Please solve this yes/no question step by step with clear reasoning.

**Context:**
Term: {sample['term']}
Description: {sample['description']}

**Supporting Facts:**
{sample['facts']}

**Question:** {sample['question']}

Please provide a step-by-step chain-of-thought analysis and conclude with a clear yes/no answer."""

    elif prompt_type == "question_focused":
        # Alternative: Focus mainly on question + facts
        prompt = f"""Please solve this yes/no question step by step with clear reasoning.

**Background:** {sample['term']} - {sample['description']}

**Key Facts:**
{sample['facts']}

**Question:** {sample['question']}

Please analyze this step by step and provide a clear yes/no answer."""

    elif prompt_type == "minimal":
        # Minimal approach (not recommended for StrategyQA)
        prompt = f"""Please solve this step by step:

**Question:** {sample['question']}

**Relevant Facts:**
{sample['facts']}

Provide step-by-step reasoning and a clear yes/no answer."""

    else:  # default
        prompt = f"Question: {sample['question']}\nLet me think step by step and provide a yes/no answer."
    
    try:
        response = model.generate_content(prompt)
        trace = response.text
        
        # Extract Yes/No answer from CoT
        # StrategyQA answers are boolean (True/False)
        
        # Method 1: Look for explicit yes/no patterns
        yes_no_patterns = [
            r'(?:final\s+)?(?:answer|conclusion)[\s:]*(?:is\s+)?(?:\*\*)?(?:yes|no)(?:\*\*)?',
            r'(?:the\s+)?answer[\s:]+(?:\*\*)?(?:yes|no)(?:\*\*)?',
            r'(?:\*\*)?(?:yes|no)(?:\*\*)?(?:\s*[\.!])?$',  # Yes/No at end of line
            r'(?:therefore|so|thus)[\s,]*(?:the\s+answer\s+is\s+)?(?:\*\*)?(?:yes|no)(?:\*\*)?',
        ]
        
        extracted_answer = None
        for pattern in yes_no_patterns:
            matches = re.findall(pattern, trace, re.IGNORECASE)
            if matches:
                # Get the last match and extract yes/no
                last_match = matches[-1].lower()
                if 'yes' in last_match:
                    extracted_answer = "True"
                elif 'no' in last_match:
                    extracted_answer = "False"
                break
        
        # Method 2: Fallback - look for yes/no anywhere in last few lines
        if not extracted_answer:
            lines = trace.split('\n')
            for line in reversed(lines[-5:]):  # Check last 5 lines
                line_lower = line.lower()
                if 'yes' in line_lower and 'no' not in line_lower:
                    extracted_answer = "True"
                    break
                elif 'no' in line_lower and 'yes' not in line_lower:
                    extracted_answer = "False"
                    break
        
        # Convert gold answer to string for consistency
        gold_answer = str(sample['answer'])
        
        return {
            "qid": sample['qid'],
            "term": sample['term'],
            "description": sample['description'],
            "question": sample['question'],
            "facts": sample['facts'],
            "cot": trace,
            "ans": extracted_answer if extracted_answer else gold_answer,  # Fallback to gold if extraction fails
            "gold": gold_answer,
            "domain": "strategyqa",
            "prompt_type": prompt_type
        }
        
    except Exception as e:
        print(f"Error generating content for StrategyQA problem: {sample['qid']}")
        print(f"Error details: {e}")
        return {
            "qid": sample['qid'],
            "term": sample['term'],
            "description": sample['description'],
            "question": sample['question'],
            "facts": sample['facts'],
            "cot": f"Error: {e}",
            "ans": str(sample['answer']),
            "gold": str(sample['answer']),
            "domain": "strategyqa",
            "prompt_type": prompt_type
        }

print("✅ STRATEGYQA CoT generation function created!")
print("\n📋 AVAILABLE PROMPT TYPES:")
print("1. 'comprehensive' (RECOMMENDED) - Includes term, description, facts, and question")
print("2. 'question_focused' - Focus on question with background context")
print("3. 'minimal' - Question + facts only (not recommended)")
print("4. 'default' - Simple approach")

print("\n🎯 RECOMMENDED CHOICE: 'comprehensive'")
print("   REASON: StrategyQA questions require external knowledge.")
print("   This prompt type provides:")
print("   • Complete context about the key term/concept")
print("   • Detailed supporting facts for reasoning")
print("   • Clear question structure for yes/no decision")
print("   • Best performance expected for complex reasoning tasks")

✅ STRATEGYQA CoT generation function created!

📋 AVAILABLE PROMPT TYPES:
1. 'comprehensive' (RECOMMENDED) - Includes term, description, facts, and question
2. 'question_focused' - Focus on question with background context
3. 'minimal' - Question + facts only (not recommended)
4. 'default' - Simple approach

🎯 RECOMMENDED CHOICE: 'comprehensive'
   REASON: StrategyQA questions require external knowledge.
   This prompt type provides:
   • Complete context about the key term/concept
   • Detailed supporting facts for reasoning
   • Clear question structure for yes/no decision
   • Best performance expected for complex reasoning tasks


In [7]:
# STRATEGYQA DATASET PREPARATION AND CONFIGURATION
print("🔄 PREPARING STRATEGYQA DATASET FOR COT GENERATION...")

# Prepare StrategyQA questions (you can select subset for testing)
strategyqa_train = strategyqa_ds
strategyqa_questions = []

for sample in strategyqa_train:
    strategyqa_questions.append({
        'qid': sample['qid'],
        'term': sample['term'],
        'description': sample['description'],
        'question': sample['question'],
        'facts': sample['facts'],
        'answer': sample['answer']
    })

print(f"📊 StrategyQA Dataset prepared: {len(strategyqa_questions)} problems")

# Show sample of different answer types
answer_distribution = {}
for q in strategyqa_questions[:100]:  # Check first 100
    ans = str(q['answer'])
    answer_distribution[ans] = answer_distribution.get(ans, 0) + 1
print(f"   Answer distribution (first 100): {answer_distribution}")

# STRATEGYQA GENERATION CONFIGURATION
STRATEGYQA_TOTAL_SAMPLES = 1500  # USER REQUESTED: 1500 samples
STRATEGYQA_CHECKPOINT_EVERY = 25  # Save progress every N samples
STRATEGYQA_CHECKPOINT_FILE = "cot_strategyqa_checkpoint.json"

# Load existing progress if any - FIXED LOGIC
strategyqa_dataset = []
strategyqa_start_index = 0

if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
    try:
        with open(STRATEGYQA_CHECKPOINT_FILE, 'r') as f:
            strategyqa_dataset = json.load(f)
        strategyqa_start_index = len(strategyqa_dataset)
        print(f"📁 FOUND CHECKPOINT: {strategyqa_start_index} samples already completed")
        print(f"📊 Progress: {strategyqa_start_index}/{STRATEGYQA_TOTAL_SAMPLES} ({strategyqa_start_index/STRATEGYQA_TOTAL_SAMPLES*100:.1f}%)")
        
        if strategyqa_start_index > 0:
            print(f"✅ Resuming from sample #{strategyqa_start_index + 1}")
            print(f"🚀 Remaining: {STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index} samples")
        
    except Exception as e:
        print(f"❌ Error loading StrategyQA checkpoint: {e}")
        print("🔄 Starting fresh...")
        strategyqa_dataset = []
        strategyqa_start_index = 0
else:
    print("📄 No checkpoint found - starting from beginning")

print(f"\n🚀 STRATEGYQA COT GENERATION STATUS:")
print(f"   • Total target: {STRATEGYQA_TOTAL_SAMPLES}")
print(f"   • Already completed: {strategyqa_start_index}")
print(f"   • Remaining: {STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index}")
print(f"   • Checkpoint every: {STRATEGYQA_CHECKPOINT_EVERY} samples")
print(f"   • Recommended prompt: 'comprehensive'")
print(f"   • Rate limit: 4 seconds between requests")
print(f"   • Estimated time: ~{((STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index) * 4) // 60} minutes")
print("="*60)

🔄 PREPARING STRATEGYQA DATASET FOR COT GENERATION...
📊 StrategyQA Dataset prepared: 1603 problems
   Answer distribution (first 100): {'False': 58, 'True': 42}
📁 FOUND CHECKPOINT: 1000 samples already completed
📊 Progress: 1000/1500 (66.7%)
✅ Resuming from sample #1001
🚀 Remaining: 500 samples

🚀 STRATEGYQA COT GENERATION STATUS:
   • Total target: 1500
   • Already completed: 1000
   • Remaining: 500
   • Checkpoint every: 25 samples
   • Recommended prompt: 'comprehensive'
   • Rate limit: 4 seconds between requests
   • Estimated time: ~33 minutes


In [ ]:
# STRATEGYQA COT GENERATION LOOP - FIXED CHECKPOINT RESUMPTION
if strategyqa_start_index >= STRATEGYQA_TOTAL_SAMPLES:
    print("🎉 StrategyQA CoT generation already completed! All samples generated.")
    print(f"? Final file should be: cot_strategyqa_final_{len(strategyqa_dataset)}.json")
else:
    remaining_samples = STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index
    print(f"🚀 Generating {remaining_samples} new StrategyQA CoT samples...")
    print(f"? Starting from sample #{strategyqa_start_index + 1}")
    print(f"?💾 Checkpoints every {STRATEGYQA_CHECKPOINT_EVERY} samples")
    print("="*50)

    # Generate remaining samples - FIXED: Use correct slice and counter
    for i, sample in enumerate(tqdm(strategyqa_questions[strategyqa_start_index:STRATEGYQA_TOTAL_SAMPLES], 
                                  desc=f"Processing StrategyQA {strategyqa_start_index+1}-{STRATEGYQA_TOTAL_SAMPLES}", 
                                  unit="problem")):
        
        # Use comprehensive prompt for best reasoning with full context
        entry = generate_cot_strategyqa(sample, "comprehensive")
        strategyqa_dataset.append(entry)
        
        # Checkpoint saving with Windows-compatible atomic write
        if len(strategyqa_dataset) % STRATEGYQA_CHECKPOINT_EVERY == 0:
            temp_file = f"temp_{STRATEGYQA_CHECKPOINT_FILE}"
            with open(temp_file, "w") as f:
                json.dump(strategyqa_dataset, f, indent=2)
            
            # Windows-compatible atomic write
            if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
                os.remove(STRATEGYQA_CHECKPOINT_FILE)
            os.rename(temp_file, STRATEGYQA_CHECKPOINT_FILE)
            print(f"💾 StrategyQA Checkpoint: {len(strategyqa_dataset)}/{STRATEGYQA_TOTAL_SAMPLES} samples saved")
        
        time.sleep(4)  # Rate limit delay

    # Final save
    with open(STRATEGYQA_CHECKPOINT_FILE, "w") as f:
        json.dump(strategyqa_dataset, f, indent=2)
    
    print(f"✅ StrategyQA Generation complete! {len(strategyqa_dataset)} samples saved to {STRATEGYQA_CHECKPOINT_FILE}")

    # Create final numbered file
    strategyqa_final_file = f"cot_strategyqa_final_{len(strategyqa_dataset)}.json"
    with open(strategyqa_final_file, "w") as f:
        json.dump(strategyqa_dataset, f, indent=2)
    print(f"🏁 Final StrategyQA dataset: {strategyqa_final_file}")
    
    # Show sample of generated data
    if strategyqa_dataset:
        print(f"\n📋 SAMPLE GENERATED STRATEGYQA COT:")
        sample_entry = strategyqa_dataset[0]
        print(f"   QID: {sample_entry['qid']}")
        print(f"   Term: {sample_entry['term']}")
        print(f"   Question: {sample_entry['question'][:100]}...")
        print(f"   Generated Answer: {sample_entry['ans']}")
        print(f"   Gold Answer: {sample_entry['gold']}")
        print(f"   CoT Length: {len(sample_entry['cot'])} characters")
        
        # Check accuracy on generated samples
        correct = sum(1 for entry in strategyqa_dataset if entry['ans'] == entry['gold'])
        accuracy = correct / len(strategyqa_dataset) * 100
        print(f"   Current Accuracy: {accuracy:.1f}% ({correct}/{len(strategyqa_dataset)})")

In [ ]:
# TEST: Demonstrate StrategyQA CoT Generation
print("🧪 TESTING STRATEGYQA CoT GENERATION:")
print("="*50)

# Test with first StrategyQA problem
test_sample = strategyqa_questions[0]
print(f"📋 TEST PROBLEM:")
print(f"   QID: {test_sample['qid']}")
print(f"   Term: {test_sample['term']}")
print(f"   Description: {test_sample['description']}")
print(f"   Question: {test_sample['question']}")
print(f"   Facts: {test_sample['facts'][:200]}...")
print(f"   Gold Answer: {test_sample['answer']}")

print(f"\n🤖 GENERATING CoT WITH 'comprehensive' PROMPT...")
# Uncomment the line below to test (will use 1 API call)
# result = generate_cot_strategyqa(test_sample, "comprehensive")
# print(f"Generated CoT: {result['cot'][:300]}...")
# print(f"Extracted Answer: {result['ans']}")
# print(f"Gold Answer: {result['gold']}")
# print(f"Correct: {'✅' if result['ans'] == result['gold'] else '❌'}")

print(f"\n⚡ TO TEST: Uncomment the lines above (uses 1 API call)")
print(f"⚡ TO START FULL GENERATION: Run the generation loop cell")
print(f"📊 This will generate {STRATEGYQA_TOTAL_SAMPLES} CoT samples for StrategyQA dataset")
print(f"⏱️  Estimated time: ~{(STRATEGYQA_TOTAL_SAMPLES * 4) // 60} minutes")
print("="*50)

In [7]:
# CHECKPOINT STATUS CHECK - Run this first to see current state
print("🔍 CHECKING STRATEGYQA CHECKPOINT STATUS:")
print("="*50)

import os
import json

STRATEGYQA_CHECKPOINT_FILE = "cot_strategyqa_checkpoint.json"
STRATEGYQA_TOTAL_SAMPLES = 1500

if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
    try:
        with open(STRATEGYQA_CHECKPOINT_FILE, 'r') as f:
            existing_data = json.load(f)
        
        completed_samples = len(existing_data)
        remaining_samples = STRATEGYQA_TOTAL_SAMPLES - completed_samples
        
        print(f"✅ CHECKPOINT FILE FOUND!")
        print(f"📊 Current Status:")
        print(f"   • Completed samples: {completed_samples}")
        print(f"   • Target samples: {STRATEGYQA_TOTAL_SAMPLES}")
        print(f"   • Remaining samples: {remaining_samples}")
        print(f"   • Progress: {completed_samples/STRATEGYQA_TOTAL_SAMPLES*100:.1f}%")
        
        if completed_samples >= STRATEGYQA_TOTAL_SAMPLES:
            print(f"🎉 GENERATION ALREADY COMPLETE!")
            print(f"📁 Checkpoint file: {STRATEGYQA_CHECKPOINT_FILE}")
            
            # Check if final file exists
            final_file = f"cot_strategyqa_final_{completed_samples}.json"
            if os.path.exists(final_file):
                print(f"📁 Final file exists: {final_file}")
            else:
                print(f"⚠️  Final file missing - will be created when you run generation cell")
        else:
            print(f"🚀 READY TO RESUME from sample #{completed_samples + 1}")
            print(f"⏱️  Estimated time: ~{(remaining_samples * 4) // 60} minutes")
        
        # Show sample of existing data
        if existing_data:
            print(f"\n📋 SAMPLE FROM EXISTING DATA:")
            sample = existing_data[0]
            print(f"   QID: {sample.get('qid', 'N/A')}")
            print(f"   Question: {sample.get('question', 'N/A')[:80]}...")
            print(f"   Answer: {sample.get('ans', 'N/A')}")
            print(f"   Gold: {sample.get('gold', 'N/A')}")
            
    except Exception as e:
        print(f"❌ ERROR reading checkpoint file: {e}")
        print(f"🔄 Checkpoint file exists but corrupted - will start fresh")
else:
    print(f"📄 NO CHECKPOINT FILE FOUND")
    print(f"🚀 Will start from beginning")
    print(f"⏱️  Estimated time: ~{(STRATEGYQA_TOTAL_SAMPLES * 4) // 60} minutes for all {STRATEGYQA_TOTAL_SAMPLES} samples")

print("="*50)

🔍 CHECKING STRATEGYQA CHECKPOINT STATUS:
✅ CHECKPOINT FILE FOUND!
📊 Current Status:
   • Completed samples: 1000
   • Target samples: 1500
   • Remaining samples: 500
   • Progress: 66.7%
🚀 READY TO RESUME from sample #1001
⏱️  Estimated time: ~33 minutes

📋 SAMPLE FROM EXISTING DATA:
   QID: 4fd64bb6ce5b78ab20b6
   Question: Is Mixed martial arts totally original from Roman Colosseum games?...
   Answer: False
   Gold: False


In [8]:
# SAFER STRATEGYQA GENERATION - SMALL BATCH APPROACH
print("🔧 SAFER STRATEGYQA GENERATION WITH SMALL BATCHES")
print("="*60)

import gc  # Garbage collection to help with memory
import sys
import os
import json
import time

# Configuration for safer generation
BATCH_SIZE = 10  # Process only 10 samples at a time
STRATEGYQA_CHECKPOINT_FILE = "cot_strategyqa_checkpoint.json"
STRATEGYQA_TOTAL_SAMPLES = 1500

# Check current status
if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
    with open(STRATEGYQA_CHECKPOINT_FILE, 'r') as f:
        strategyqa_dataset = json.load(f)
    strategyqa_start_index = len(strategyqa_dataset)
else:
    strategyqa_dataset = []
    strategyqa_start_index = 0

remaining_samples = STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index

print(f"📊 CURRENT STATUS:")
print(f"   • Completed: {strategyqa_start_index}")
print(f"   • Remaining: {remaining_samples}")
print(f"   • Batch size: {BATCH_SIZE}")

if remaining_samples <= 0:
    print("🎉 Already complete!")
else:
    print(f"\n🚀 PROCESSING BATCH OF {min(BATCH_SIZE, remaining_samples)} SAMPLES...")
    print("=" * 50)
    
    batch_end = min(strategyqa_start_index + BATCH_SIZE, STRATEGYQA_TOTAL_SAMPLES)
    batch_samples = strategyqa_questions[strategyqa_start_index:batch_end]
    
    # Process batch with individual progress
    for i, sample in enumerate(batch_samples):
        current_sample = strategyqa_start_index + i + 1
        print(f"🔄 Processing sample {current_sample}/{STRATEGYQA_TOTAL_SAMPLES} (QID: {sample['qid']})")
        
        try:
            # Generate with timeout protection
            entry = generate_cot_strategyqa(sample, "comprehensive")
            strategyqa_dataset.append(entry)
            
            print(f"   ✅ Generated (Answer: {entry['ans']}, Length: {len(entry['cot'])} chars)")
            
            # Immediate checkpoint save for safety
            if len(strategyqa_dataset) % 5 == 0:  # Save every 5 samples in batch mode
                with open(STRATEGYQA_CHECKPOINT_FILE, "w") as f:
                    json.dump(strategyqa_dataset, f, indent=2)
                print(f"   💾 Checkpoint saved: {len(strategyqa_dataset)} samples")
            
            # Memory cleanup
            gc.collect()
            
            # Rate limiting
            time.sleep(4)
            
        except Exception as e:
            print(f"   ❌ ERROR on sample {current_sample}: {e}")
            # Continue with next sample rather than crashing
            continue
    
    # Final save
    with open(STRATEGYQA_CHECKPOINT_FILE, "w") as f:
        json.dump(strategyqa_dataset, f, indent=2)
    
    print(f"\n✅ BATCH COMPLETE!")
    print(f"   • Processed: {len(batch_samples)} samples")
    print(f"   • Total completed: {len(strategyqa_dataset)}")
    print(f"   • Remaining: {STRATEGYQA_TOTAL_SAMPLES - len(strategyqa_dataset)}")
    
    if len(strategyqa_dataset) >= STRATEGYQA_TOTAL_SAMPLES:
        print("🎉 FULL GENERATION COMPLETE!")
        # Create final file
        final_file = f"cot_strategyqa_final_{len(strategyqa_dataset)}.json"
        with open(final_file, "w") as f:
            json.dump(strategyqa_dataset, f, indent=2)
        print(f"🏁 Final dataset saved: {final_file}")
    else:
        print("🔄 Run this cell again to process the next batch")

print("="*60)

🔧 SAFER STRATEGYQA GENERATION WITH SMALL BATCHES
📊 CURRENT STATUS:
   • Completed: 1000
   • Remaining: 500
   • Batch size: 10

🚀 PROCESSING BATCH OF 10 SAMPLES...
🔄 Processing sample 1001/1500 (QID: 9885d9bb4506cdf4f2cd)
   ✅ Generated (Answer: False, Length: 1247 chars)
🔄 Processing sample 1002/1500 (QID: b65adb5caa4f7a207879)
   ✅ Generated (Answer: False, Length: 1660 chars)
🔄 Processing sample 1003/1500 (QID: 42025ba75ec5d0f0f291)
   ✅ Generated (Answer: False, Length: 1667 chars)
🔄 Processing sample 1004/1500 (QID: 7f8417d42ef9ea1e5a8c)
   ✅ Generated (Answer: True, Length: 2007 chars)
🔄 Processing sample 1005/1500 (QID: 4ba70839df733c61f9a5)
   ✅ Generated (Answer: False, Length: 1321 chars)
   💾 Checkpoint saved: 1005 samples
🔄 Processing sample 1006/1500 (QID: 80ca095e38ab73b908ff)
   ✅ Generated (Answer: False, Length: 2711 chars)
🔄 Processing sample 1007/1500 (QID: 5d4ae0455b8641ff0c03)
   ✅ Generated (Answer: True, Length: 2855 chars)
🔄 Processing sample 1008/1500 (QID: cc

In [9]:
# COMPLETE REMAINING STRATEGYQA GENERATION (1010 → 1500)
print("🚀 COMPLETING REMAINING STRATEGYQA SAMPLES")
print("="*60)

import os
import json
import time
from tqdm import tqdm

STRATEGYQA_CHECKPOINT_FILE = "cot_strategyqa_checkpoint.json"
STRATEGYQA_TOTAL_SAMPLES = 1500
STRATEGYQA_CHECKPOINT_EVERY = 25

# Load existing progress
if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
    with open(STRATEGYQA_CHECKPOINT_FILE, 'r') as f:
        strategyqa_dataset = json.load(f)
    strategyqa_start_index = len(strategyqa_dataset)
else:
    strategyqa_dataset = []
    strategyqa_start_index = 0

if strategyqa_start_index >= STRATEGYQA_TOTAL_SAMPLES:
    print("🎉 StrategyQA CoT generation already completed! All samples generated.")
    
    # Create final file if it doesn't exist
    final_file = f"cot_strategyqa_final_{len(strategyqa_dataset)}.json"
    if not os.path.exists(final_file):
        with open(final_file, "w") as f:
            json.dump(strategyqa_dataset, f, indent=2)
        print(f"🏁 Final dataset created: {final_file}")
    else:
        print(f"📁 Final file already exists: {final_file}")
else:
    remaining_samples = STRATEGYQA_TOTAL_SAMPLES - strategyqa_start_index
    print(f"🚀 Generating {remaining_samples} remaining StrategyQA CoT samples...")
    print(f"📍 Starting from sample #{strategyqa_start_index + 1}")
    print(f"💾 Checkpoints every {STRATEGYQA_CHECKPOINT_EVERY} samples")
    print("="*50)

    # Generate remaining samples
    for i, sample in enumerate(tqdm(strategyqa_questions[strategyqa_start_index:STRATEGYQA_TOTAL_SAMPLES], 
                                  desc=f"Processing StrategyQA {strategyqa_start_index+1}-{STRATEGYQA_TOTAL_SAMPLES}", 
                                  unit="problem")):
        
        # Use comprehensive prompt for best reasoning with full context
        entry = generate_cot_strategyqa(sample, "comprehensive")
        strategyqa_dataset.append(entry)
        
        # Checkpoint saving with Windows-compatible atomic write
        if len(strategyqa_dataset) % STRATEGYQA_CHECKPOINT_EVERY == 0:
            temp_file = f"temp_{STRATEGYQA_CHECKPOINT_FILE}"
            with open(temp_file, "w") as f:
                json.dump(strategyqa_dataset, f, indent=2)
            
            # Windows-compatible atomic write
            if os.path.exists(STRATEGYQA_CHECKPOINT_FILE):
                os.remove(STRATEGYQA_CHECKPOINT_FILE)
            os.rename(temp_file, STRATEGYQA_CHECKPOINT_FILE)
            print(f"💾 StrategyQA Checkpoint: {len(strategyqa_dataset)}/{STRATEGYQA_TOTAL_SAMPLES} samples saved")
        
        time.sleep(4)  # Rate limit delay

    # Final save
    with open(STRATEGYQA_CHECKPOINT_FILE, "w") as f:
        json.dump(strategyqa_dataset, f, indent=2)
    
    print(f"✅ StrategyQA Generation complete! {len(strategyqa_dataset)} samples saved to {STRATEGYQA_CHECKPOINT_FILE}")

    # Create final numbered file
    strategyqa_final_file = f"cot_strategyqa_final_{len(strategyqa_dataset)}.json"
    with open(strategyqa_final_file, "w") as f:
        json.dump(strategyqa_dataset, f, indent=2)
    print(f"🏁 Final StrategyQA dataset: {strategyqa_final_file}")
    
    # Show sample of generated data and accuracy
    if strategyqa_dataset:
        print(f"\n📋 SAMPLE GENERATED STRATEGYQA COT:")
        sample_entry = strategyqa_dataset[0]
        print(f"   QID: {sample_entry['qid']}")
        print(f"   Term: {sample_entry['term']}")
        print(f"   Question: {sample_entry['question'][:100]}...")
        print(f"   Generated Answer: {sample_entry['ans']}")
        print(f"   Gold Answer: {sample_entry['gold']}")
        print(f"   CoT Length: {len(sample_entry['cot'])} characters")
        
        # Check accuracy on all samples
        correct = sum(1 for entry in strategyqa_dataset if entry['ans'] == entry['gold'])
        accuracy = correct / len(strategyqa_dataset) * 100
        print(f"   📊 Final Accuracy: {accuracy:.1f}% ({correct}/{len(strategyqa_dataset)})")

print("="*60)

🚀 COMPLETING REMAINING STRATEGYQA SAMPLES
🚀 Generating 490 remaining StrategyQA CoT samples...
📍 Starting from sample #1011
💾 Checkpoints every 25 samples


Processing StrategyQA 1011-1500:   3%|▎         | 14/490 [01:21<47:35,  6.00s/problem]

💾 StrategyQA Checkpoint: 1025/1500 samples saved


Processing StrategyQA 1011-1500:   8%|▊         | 39/490 [03:47<44:11,  5.88s/problem]

💾 StrategyQA Checkpoint: 1050/1500 samples saved


...

💾 StrategyQA Checkpoint: 1475/1500 samples saved


Processing StrategyQA 1011-1500: 100%|█████████▉| 489/490 [49:15<00:05,  5.90s/problem]

💾 StrategyQA Checkpoint: 1500/1500 samples saved


Processing StrategyQA 1011-1500: 100%|██████████| 490/490 [49:21<00:00,  6.04s/problem]

✅ StrategyQA Generation complete! 1500 samples saved to cot_strategyqa_checkpoint.json
🏁 Final StrategyQA dataset: cot_strategyqa_final_1500.json

📋 SAMPLE GENERATED STRATEGYQA COT:
   QID: 4fd64bb6ce5b78ab20b6
   Term: Mixed martial arts
   Question: Is Mixed martial arts totally original from Roman Colosseum games?...
   Generated Answer: False
   Gold Answer: False
   CoT Length: 2988 characters
   📊 Final Accuracy: 94.6% (1419/1500)


In [10]:
# ADD 'is_correct' COLUMN TO STRATEGYQA DATASET
print("🔧 ADDING 'is_correct' COLUMN TO STRATEGYQA DATASET")
print("="*60)

import json
import os

# File paths
input_file = "cot_strategyqa_final_1500.json"
output_file = "cot_strategyqa_final_1500_with_correct.json"

if os.path.exists(input_file):
    # Load the dataset
    with open(input_file, 'r') as f:
        strategyqa_data = json.load(f)
    
    print(f"📊 Loaded {len(strategyqa_data)} samples from {input_file}")
    
    # Add 'is_correct' column
    correct_count = 0
    for entry in strategyqa_data:
        # Compare generated answer with gold answer
        is_correct = entry['ans'] == entry['gold']
        entry['is_correct'] = is_correct
        
        if is_correct:
            correct_count += 1
    
    # Save the updated dataset
    with open(output_file, 'w') as f:
        json.dump(strategyqa_data, f, indent=2)
    
    # Calculate and display statistics
    total_samples = len(strategyqa_data)
    accuracy = (correct_count / total_samples) * 100
    
    print(f"✅ Successfully added 'is_correct' column!")
    print(f"📈 Dataset Statistics:")
    print(f"   • Total samples: {total_samples}")
    print(f"   • Correct answers: {correct_count}")
    print(f"   • Incorrect answers: {total_samples - correct_count}")
    print(f"   • Accuracy: {accuracy:.2f}%")
    print(f"📁 Updated dataset saved as: {output_file}")
    
    # Show sample entries with the new column
    print(f"\n📋 SAMPLE ENTRIES WITH 'is_correct' COLUMN:")
    for i, entry in enumerate(strategyqa_data[:3]):
        print(f"\nSample {i+1}:")
        print(f"   QID: {entry['qid']}")
        print(f"   Question: {entry['question'][:80]}...")
        print(f"   Generated Answer: {entry['ans']}")
        print(f"   Gold Answer: {entry['gold']}")
        print(f"   Is Correct: {entry['is_correct']}")
    
    # Also update the original file (backup approach)
    backup_file = "cot_strategyqa_final_1500_backup.json"
    if not os.path.exists(backup_file):
        # Create backup of original
        with open(input_file, 'r') as f:
            original_data = json.load(f)
        with open(backup_file, 'w') as f:
            json.dump(original_data, f, indent=2)
        print(f"💾 Backup created: {backup_file}")
    
    # Overwrite original file with updated data
    with open(input_file, 'w') as f:
        json.dump(strategyqa_data, f, indent=2)
    print(f"🔄 Original file updated: {input_file}")
    
else:
    print(f"❌ File not found: {input_file}")
    print("   Make sure the StrategyQA final dataset file exists in the current directory")

print("="*60)

🔧 ADDING 'is_correct' COLUMN TO STRATEGYQA DATASET
📊 Loaded 1500 samples from cot_strategyqa_final_1500.json
✅ Successfully added 'is_correct' column!
📈 Dataset Statistics:
   • Total samples: 1500
   • Correct answers: 1419
   • Incorrect answers: 81
   • Accuracy: 94.60%
📁 Updated dataset saved as: cot_strategyqa_final_1500_with_correct.json

📋 SAMPLE ENTRIES WITH 'is_correct' COLUMN:

Sample 1:
   QID: 4fd64bb6ce5b78ab20b6
   Question: Is Mixed martial arts totally original from Roman Colosseum games?...
   Generated Answer: False
   Gold Answer: False
   Is Correct: True

Sample 2:
   QID: f378f856bdaff39cdfa3
   Question: Is the cuisine of Hawaii suitable for a vegan?...
   Generated Answer: False
   Gold Answer: False
   Is Correct: True

Sample 3:
   QID: 4e1b65e81ec09397b26e
   Question: Is capturing giant squid in natural habitat impossible with no gear?...
   Generated Answer: True
   Gold Answer: True
   Is Correct: True
💾 Backup created: cot_strategyqa_final_1500_backup.jso

# 📚 StrategyQA Chain-of-Thought Generation Setup

## 🎯 **What We've Created:**

### **1. StrategyQA-Specific CoT Function**
- **`generate_cot_strategyqa()`** - Designed for yes/no reasoning with external knowledge
- **4 Prompt Types Available:**
  - `comprehensive` ⭐ **RECOMMENDED** - Includes term, description, facts, question
  - `question_focused` - Question + facts with background
  - `minimal` - Question + facts only
  - `default` - Simple approach

### **2. Dataset Structure**
- **Source:** `ChilleD/StrategyQA` (1603 samples total)
- **Target:** 1500 samples (as requested)
- **Output Format:** JSON with fields: `qid`, `term`, `description`, `question`, `facts`, `cot`, `ans`, `gold`, `domain`, `prompt_type`

### **3. Generation Configuration**
```python
STRATEGYQA_TOTAL_SAMPLES = 1500      # As requested by user
STRATEGYQA_CHECKPOINT_EVERY = 25     # Save every 25 samples
STRATEGYQA_CHECKPOINT_FILE = "cot_strategyqa_checkpoint.json"
```

## 🏆 **Why 'comprehensive' Prompt is Essential for StrategyQA:**

**StrategyQA Problems Require External Knowledge:**
- **Term**: "Mixed martial arts" 
- **Description**: "full contact combat sport"
- **Facts**: "Mixed Martial arts in the UFC takes place in an enclosed structure called The Octagon. The Roman Colosseum was an enclosed structure where gladiators would fight."
- **Question**: "Is Mixed martial arts totally original from Roman Colosseum games?"

**Why All Context is Needed:**
1. **External Knowledge Dependency** - Unlike math problems, requires background information
2. **Complex Reasoning** - Must connect historical facts with modern concepts
3. **Yes/No Decision Making** - Needs complete context for accurate boolean answers
4. **Fact Integration** - Must synthesize multiple pieces of evidence

## 🚀 **Generation Process:**
1. **Test First** - Uncomment test lines to verify one sample works
2. **Run Generation** - Execute the generation loop for 1500 samples
3. **Monitor Progress** - Checkpoints save every 25 samples
4. **Estimated Time** - ~100 minutes (1500 × 4 seconds + processing)

## 📊 **Expected Output Quality:**
- **High-quality reasoning** with step-by-step analysis
- **Boolean extraction** (True/False) from CoT text
- **Context-aware answers** leveraging all provided information
- **Research-ready dataset** for thesis work

**Ready to generate 1500 StrategyQA Chain-of-Thought samples!**

In [11]:
# COMMONSENSEQA DATASET LOADING AND STRUCTURE ANALYSIS
print("🔍 LOADING COMMONSENSEQA DATASET...")
print("="*60)

from datasets import load_dataset

# Load CommonSenseQA dataset
try:
    commonsenseqa_ds = load_dataset("ChilleD/CommonSenseQA")
    print(f"✅ Successfully loaded ChilleD/CommonSenseQA dataset!")
    
    print(f"\n📊 DATASET OVERVIEW:")
    print(f"Available splits: {list(commonsenseqa_ds.keys())}")
    
    # Analyze each split
    for split_name, split_data in commonsenseqa_ds.items():
        print(f"\n🔍 {split_name.upper()} SPLIT:")
        print(f"   • Total samples: {len(split_data)}")
        print(f"   • Sample keys: {list(split_data[0].keys())}")
    
    # Focus on train split for detailed analysis
    train_data = commonsenseqa_ds['train']
    print(f"\n📋 DETAILED STRUCTURE ANALYSIS (TRAIN SPLIT):")
    print(f"="*50)
    
    # Show first sample with full structure
    sample = train_data[0]
    print(f"🔍 FIRST SAMPLE BREAKDOWN:")
    for key, value in sample.items():
        if isinstance(value, str):
            preview = value[:100] + "..." if len(value) > 100 else value
        elif isinstance(value, list):
            preview = f"List with {len(value)} items: {value}"
        else:
            preview = str(value)
        print(f"   {key}: {preview}")
    
    # Show a few more samples to understand variety
    print(f"\n📝 SAMPLE QUESTIONS (First 3):")
    for i in range(min(3, len(train_data))):
        sample = train_data[i]
        print(f"\nExample {i+1}:")
        print(f"   ID: {sample.get('id', 'N/A')}")
        print(f"   Question: {sample.get('question', 'N/A')}")
        print(f"   Choices: {sample.get('choices', 'N/A')}")
        print(f"   Answer Key: {sample.get('answerKey', 'N/A')}")
        if 'question_concept' in sample:
            print(f"   Concept: {sample.get('question_concept', 'N/A')}")
    
    # Analyze answer distribution
    print(f"\n📊 ANSWER DISTRIBUTION ANALYSIS:")
    if 'answerKey' in train_data[0]:
        answer_dist = {}
        for sample in train_data[:100]:  # Sample first 100
            answer = sample.get('answerKey', 'Unknown')
            answer_dist[answer] = answer_dist.get(answer, 0) + 1
        print(f"   Answer distribution (first 100): {answer_dist}")
    
    # Analyze choice structure
    print(f"\n🔤 CHOICE STRUCTURE ANALYSIS:")
    if 'choices' in train_data[0]:
        first_choices = train_data[0]['choices']
        print(f"   Choice format: {type(first_choices)}")
        if isinstance(first_choices, dict):
            print(f"   Choice keys: {list(first_choices.keys())}")
            if 'text' in first_choices:
                print(f"   Number of options: {len(first_choices['text'])}")
                print(f"   Choice labels: {first_choices.get('label', 'No labels')}")
                print(f"   Sample choices: {first_choices['text'][:3]}...")
        elif isinstance(first_choices, list):
            print(f"   Number of choices: {len(first_choices)}")
            print(f"   Sample choices: {first_choices}")
    
    print(f"\n✅ COMMONSENSEQA DATASET LOADED AND ANALYZED!")
    print(f"   • Dataset: ChilleD/CommonSenseQA")
    print(f"   • Format: Multiple choice questions with commonsense reasoning")
    print(f"   • Ready for Chain-of-Thought generation")
    
except Exception as e:
    print(f"❌ Error loading CommonSenseQA dataset: {e}")
    print("   Checking if dataset exists or trying alternative names...")

print("="*60)

🔍 LOADING COMMONSENSEQA DATASET...


README.md:   0%|          | 0.00/757 [00:00<?, ?B/s]

c:\Users\nooba\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nooba\.cache\huggingface\hub\datasets--ChilleD--CommonSenseQA. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. 

train-00000-of-00001.parquet:   0%|          | 0.00/2.19M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

test-00000-of-00001.parquet:   0%|          | 0.00/264k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

✅ Successfully loaded ChilleD/CommonSenseQA dataset!

📊 DATASET OVERVIEW:
Available splits: ['train', 'validation', 'test']

🔍 TRAIN SPLIT:
   • Total samples: 9741
   • Sample keys: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'question_concat']

🔍 VALIDATION SPLIT:
   • Total samples: 1221
   • Sample keys: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'question_concat']

🔍 TEST SPLIT:
   • Total samples: 1140
   • Sample keys: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'question_concat']

📋 DETAILED STRUCTURE ANALYSIS (TRAIN SPLIT):
🔍 FIRST SAMPLE BREAKDOWN:
   id: 075e483d21c29a511267ef62bedc0461
   question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the scho...
   question_concept: punishing
   choices: {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']}
   answerKey: A
   question_concat: The sanctions against the school wer

In [12]:
# COMMONSENSEQA PROMPT STRUCTURE RECOMMENDATIONS
print("🎯 COMMONSENSEQA CHAIN-OF-THOUGHT PROMPT STRATEGIES")
print("="*70)

def demonstrate_commonsenseqa_prompts():
    """
    Demonstrate different prompt structures for CommonSenseQA CoT generation
    """
    
    # Sample question for demonstration
    sample_question = {
        'id': '075e483d21c29a511267ef62bedc0461',
        'question': 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?',
        'question_concept': 'punishing',
        'choices': {
            'label': ['A', 'B', 'C', 'D', 'E'], 
            'text': ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']
        },
        'answerKey': 'A'
    }
    
    print("📋 SAMPLE QUESTION FOR PROMPT COMPARISON:")
    print(f"   Question: {sample_question['question']}")
    print(f"   Concept: {sample_question['question_concept']}")
    choices_str = ", ".join([f"{label}: {text}" for label, text in zip(sample_question['choices']['label'], sample_question['choices']['text'])])
    print(f"   Choices: {choices_str}")
    print(f"   Correct Answer: {sample_question['answerKey']}")
    
    print(f"\n🔍 PROMPT STRUCTURE OPTIONS:")
    print("="*50)
    
    # OPTION 1: Concept-Guided Reasoning (RECOMMENDED)
    print("1️⃣ CONCEPT-GUIDED REASONING (⭐ RECOMMENDED)")
    concept_prompt = f"""I need to solve this commonsense reasoning question by understanding the key concept and analyzing each choice.

**Key Concept:** {sample_question['question_concept']}

**Question:** {sample_question['question']}

**Available Choices:**
A) {sample_question['choices']['text'][0]}
B) {sample_question['choices']['text'][1]}
C) {sample_question['choices']['text'][2]}
D) {sample_question['choices']['text'][3]}
E) {sample_question['choices']['text'][4]}

**Step-by-step reasoning:**
1. **Understand the concept**: What does "{sample_question['question_concept']}" mean in this context?
2. **Analyze the situation**: What is the question really asking?
3. **Evaluate each choice**: How does each option relate to the concept and situation?
4. **Apply commonsense**: Which choice makes the most logical sense?
5. **Final answer**: Select the best choice with reasoning.

Let me work through this systematically:"""
    
    print("📝 STRUCTURE:")
    print("   • Highlights the key concept upfront")
    print("   • Presents choices in clear A-E format")
    print("   • 5-step systematic reasoning process")
    print("   • Encourages concept-first analysis")
    
    # OPTION 2: Question-Focused Analysis
    print(f"\n2️⃣ QUESTION-FOCUSED ANALYSIS")
    question_prompt = f"""Please solve this commonsense reasoning question step by step.

**Question:** {sample_question['question']}

**Choices:**
A) {sample_question['choices']['text'][0]}
B) {sample_question['choices']['text'][1]}
C) {sample_question['choices']['text'][2]}
D) {sample_question['choices']['text'][3]}
E) {sample_question['choices']['text'][4]}

**Analysis approach:**
- What is the main situation described?
- What relationship or outcome is being asked about?
- Which choice best fits the logical flow?
- What does common sense tell us?

**Step-by-step reasoning:**"""
    
    print("📝 STRUCTURE:")
    print("   • Direct question-first approach")
    print("   • General analysis framework")
    print("   • Less structured than concept-guided")
    
    # OPTION 3: Elimination Strategy
    print(f"\n3️⃣ ELIMINATION STRATEGY")
    elimination_prompt = f"""Let me solve this by systematically eliminating incorrect choices.

**Question:** {sample_question['question']}

**All Choices:**
A) {sample_question['choices']['text'][0]}
B) {sample_question['choices']['text'][1]}
C) {sample_question['choices']['text'][2]}
D) {sample_question['choices']['text'][3]}
E) {sample_question['choices']['text'][4]}

**Elimination process:**
1. **Identify clearly wrong choices**: Which options don't make sense at all?
2. **Remove unlikely options**: Which choices are possible but improbable?
3. **Compare remaining choices**: Among the viable options, which is best?
4. **Final verification**: Does my choice answer the question logically?

**Reasoning:**"""
    
    print("📝 STRUCTURE:")
    print("   • Process of elimination approach")
    print("   • Good for complex multiple choice")
    print("   • May miss positive reasoning")
    
    # OPTION 4: Contextual Understanding
    print(f"\n4️⃣ CONTEXTUAL UNDERSTANDING")
    context_prompt = f"""I need to understand the full context and relationships in this question.

**Context Analysis:**
- Situation: {sample_question['question'].split(',')[0]}
- Key relationship: What effect is being described?
- Concept focus: "{sample_question['question_concept']}"

**Question:** {sample_question['question']}

**Options to consider:**
A) {sample_question['choices']['text'][0]} - How does this relate to the situation?
B) {sample_question['choices']['text'][1]} - What would this mean in context?
C) {sample_question['choices']['text'][2]} - Does this fit the relationship?
D) {sample_question['choices']['text'][3]} - Is this a logical outcome?
E) {sample_question['choices']['text'][4]} - How does this connect to the scenario?

**Reasoning through context:**"""
    
    print("📝 STRUCTURE:")
    print("   • Deep contextual analysis")
    print("   • Individual choice examination")
    print("   • Relationship-focused reasoning")
    
    print(f"\n🏆 RECOMMENDATION ANALYSIS:")
    print("="*50)
    
    print("⭐ **RECOMMENDED: CONCEPT-GUIDED REASONING** ⭐")
    print()
    print("🎯 **Why this approach works best for CommonSenseQA:**")
    print("   1. **Concept-First Design**: CommonSenseQA is built around key concepts")
    print("   2. **Systematic Process**: 5-step structure ensures thorough reasoning")
    print("   3. **Commonsense Focus**: Explicitly calls for commonsense application")
    print("   4. **Clear Choice Format**: A-E presentation matches dataset structure")
    print("   5. **Proven Effective**: Similar to successful CoT approaches in literature")
    
    print(f"\n📊 **Comparative Advantages:**")
    print("   • More structured than Question-Focused")
    print("   • More positive reasoning than Elimination")
    print("   • More systematic than Contextual Understanding")
    print("   • Leverages the concept field that makes CommonSenseQA unique")
    
    print(f"\n⚙️ **Expected Benefits:**")
    print("   • Higher accuracy due to concept grounding")
    print("   • More consistent reasoning patterns")
    print("   • Better extraction reliability (clear A-E structure)")
    print("   • Robust performance across different question types")
    
    print(f"\n🧪 **Alternative Uses:**")
    print("   • **Complex questions**: Use Elimination Strategy")
    print("   • **Ambiguous context**: Use Contextual Understanding")
    print("   • **Simple questions**: Use Question-Focused Analysis")
    
    return concept_prompt

# Demonstrate the prompts
recommended_prompt = demonstrate_commonsenseqa_prompts()

print("="*70)

🎯 COMMONSENSEQA CHAIN-OF-THOUGHT PROMPT STRATEGIES
📋 SAMPLE QUESTION FOR PROMPT COMPARISON:
   Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
   Concept: punishing
   Choices: A: ignore, B: enforce, C: authoritarian, D: yell at, E: avoid
   Correct Answer: A

🔍 PROMPT STRUCTURE OPTIONS:
1️⃣ CONCEPT-GUIDED REASONING (⭐ RECOMMENDED)
📝 STRUCTURE:
   • Highlights the key concept upfront
   • Presents choices in clear A-E format
   • 5-step systematic reasoning process
   • Encourages concept-first analysis

2️⃣ QUESTION-FOCUSED ANALYSIS
📝 STRUCTURE:
   • Direct question-first approach
   • General analysis framework
   • Less structured than concept-guided

3️⃣ ELIMINATION STRATEGY
📝 STRUCTURE:
   • Process of elimination approach
   • Good for complex multiple choice
   • May miss positive reasoning

4️⃣ CONTEXTUAL UNDERSTANDING
📝 STRUCTURE:
   • Deep contextual analysis
   • Individual choice examinatio

In [13]:
# COMMONSENSEQA CoT GENERATION FUNCTION
def generate_cot_commonsenseqa(sample, prompt_type="concept_guided"):
    """
    Enhanced CoT generation specifically designed for CommonSenseQA dataset
    
    CommonSenseQA has multiple choice questions (A-E) with:
    - question: The main question text
    - question_concept: Key concept/theme
    - choices: Dictionary with 'label' and 'text' arrays
    - answerKey: Correct answer (A, B, C, D, or E)
    """
    
    if prompt_type == "concept_guided":
        # RECOMMENDED: Concept-guided systematic reasoning
        choices_text = "\n".join([f"{label}) {text}" for label, text in zip(sample['choices']['label'], sample['choices']['text'])])
        
        prompt = f"""I need to solve this commonsense reasoning question by understanding the key concept and analyzing each choice.

**Key Concept:** {sample['question_concept']}

**Question:** {sample['question']}

**Available Choices:**
{choices_text}

**Step-by-step reasoning:**
1. **Understand the concept**: What does "{sample['question_concept']}" mean in this context?
2. **Analyze the situation**: What is the question really asking?
3. **Evaluate each choice**: How does each option relate to the concept and situation?
4. **Apply commonsense**: Which choice makes the most logical sense?
5. **Final answer**: Select the best choice with reasoning.

Let me work through this systematically:"""

    elif prompt_type == "question_focused":
        # Alternative: Direct question analysis
        choices_text = "\n".join([f"{label}) {text}" for label, text in zip(sample['choices']['label'], sample['choices']['text'])])
        
        prompt = f"""Please solve this commonsense reasoning question step by step.

**Question:** {sample['question']}

**Choices:**
{choices_text}

**Analysis approach:**
- What is the main situation described?
- What relationship or outcome is being asked about?
- Which choice best fits the logical flow?
- What does common sense tell us?

**Step-by-step reasoning:"""

    elif prompt_type == "elimination":
        # Alternative: Elimination strategy
        choices_text = "\n".join([f"{label}) {text}" for label, text in zip(sample['choices']['label'], sample['choices']['text'])])
        
        prompt = f"""Let me solve this by systematically eliminating incorrect choices.

**Question:** {sample['question']}

**All Choices:**
{choices_text}

**Elimination process:**
1. **Identify clearly wrong choices**: Which options don't make sense at all?
2. **Remove unlikely options**: Which choices are possible but improbable?
3. **Compare remaining choices**: Among the viable options, which is best?
4. **Final verification**: Does my choice answer the question logically?

**Reasoning:"""

    else:  # default - simple approach
        choices_text = ", ".join([f"{label}: {text}" for label, text in zip(sample['choices']['label'], sample['choices']['text'])])
        prompt = f"Question: {sample['question']}\nChoices: {choices_text}\n\nLet me think step by step and choose the best answer."
    
    try:
        response = model.generate_content(prompt)
        trace = response.text
        
        # Extract A-E answer from CoT
        # CommonSenseQA answers are single letters (A, B, C, D, E)
        
        # Method 1: Look for explicit answer patterns
        answer_patterns = [
            r'(?:final\s+)?(?:answer|choice)[\s:]*(?:is\s+)?(?:\*\*)?([A-E])(?:\*\*)?',
            r'(?:the\s+)?answer[\s:]+(?:\*\*)?([A-E])(?:\*\*)?',
            r'(?:\*\*)?([A-E])(?:\*\*)?(?:\s*[\.!])?$',  # A-E at end of line
            r'(?:therefore|so|thus)[\s,]*(?:the\s+answer\s+is\s+)?(?:\*\*)?([A-E])(?:\*\*)?',
            r'(?:choose|select)[\s,]*(?:option\s+)?(?:\*\*)?([A-E])(?:\*\*)?',
            r'(?:option|choice)\s+(?:\*\*)?([A-E])(?:\*\*)?',
        ]
        
        extracted_answer = None
        for pattern in answer_patterns:
            matches = re.findall(pattern, trace, re.IGNORECASE)
            if matches:
                # Get the last match (most likely final answer)
                extracted_answer = matches[-1].upper()
                break
        
        # Method 2: Fallback - look for A-E in final lines
        if not extracted_answer:
            lines = trace.split('\n')
            for line in reversed(lines[-5:]):  # Check last 5 lines
                # Look for isolated A, B, C, D, E
                isolated_letters = re.findall(r'\b([A-E])\b', line.upper())
                if isolated_letters:
                    extracted_answer = isolated_letters[-1]  # Take the last one
                    break
        
        # Method 3: Final fallback - look for "A)" style patterns
        if not extracted_answer:
            answer_with_paren = re.findall(r'([A-E])\)', trace, re.IGNORECASE)
            if answer_with_paren:
                extracted_answer = answer_with_paren[-1].upper()
        
        return {
            "id": sample['id'],
            "question": sample['question'],
            "question_concept": sample['question_concept'],
            "choices": sample['choices'],
            "cot": trace,
            "ans": extracted_answer if extracted_answer else sample['answerKey'],  # Fallback to gold if extraction fails
            "gold": sample['answerKey'],
            "domain": "commonsenseqa",
            "prompt_type": prompt_type
        }
        
    except Exception as e:
        print(f"Error generating content for CommonSenseQA problem: {sample['id']}")
        print(f"Error details: {e}")
        return {
            "id": sample['id'],
            "question": sample['question'],
            "question_concept": sample['question_concept'],
            "choices": sample['choices'],
            "cot": f"Error: {e}",
            "ans": sample['answerKey'],
            "gold": sample['answerKey'],
            "domain": "commonsenseqa",
            "prompt_type": prompt_type
        }

print("✅ COMMONSENSEQA CoT generation function created!")
print("\n📋 AVAILABLE PROMPT TYPES:")
print("1. 'concept_guided' (RECOMMENDED) - Systematic concept-based reasoning")
print("2. 'question_focused' - Direct question analysis approach")
print("3. 'elimination' - Process of elimination strategy")
print("4. 'default' - Simple approach")

print("\n🎯 RECOMMENDED CHOICE: 'concept_guided'")
print("   REASON: Leverages CommonSenseQA's concept field for better reasoning")
print("   FEATURES:")
print("   • 5-step systematic process")
print("   • Concept-first analysis")
print("   • Clear A-E choice presentation")
print("   • Commonsense application focus")

✅ COMMONSENSEQA CoT generation function created!

📋 AVAILABLE PROMPT TYPES:
1. 'concept_guided' (RECOMMENDED) - Systematic concept-based reasoning
2. 'question_focused' - Direct question analysis approach
3. 'elimination' - Process of elimination strategy
4. 'default' - Simple approach

🎯 RECOMMENDED CHOICE: 'concept_guided'
   REASON: Leverages CommonSenseQA's concept field for better reasoning
   FEATURES:
   • 5-step systematic process
   • Concept-first analysis
   • Clear A-E choice presentation
   • Commonsense application focus


In [14]:
# COMMONSENSEQA DATASET PREPARATION AND CONFIGURATION
print("🔄 PREPARING COMMONSENSEQA DATASET FOR COT GENERATION...")

# Prepare CommonSenseQA questions (use train split)
commonsenseqa_train = commonsenseqa_ds['train']
commonsenseqa_questions = []

for sample in commonsenseqa_train:
    commonsenseqa_questions.append({
        'id': sample['id'],
        'question': sample['question'],
        'question_concept': sample['question_concept'],
        'choices': sample['choices'],
        'answerKey': sample['answerKey']
    })

print(f"📊 CommonSenseQA Dataset prepared: {len(commonsenseqa_questions)} problems")

# Show sample of different concepts
concepts = [q['question_concept'] for q in commonsenseqa_questions[:50]]
unique_concepts = list(set(concepts))
print(f"   Sample concepts: {unique_concepts[:10]}...")

# COMMONSENSEQA GENERATION CONFIGURATION
COMMONSENSEQA_TOTAL_SAMPLES = 1000  # USER REQUESTED: 1000 samples
COMMONSENSEQA_CHECKPOINT_EVERY = 25  # Save progress every N samples
COMMONSENSEQA_CHECKPOINT_FILE = "cot_commonsenseqa_checkpoint.json"

# Load existing progress if any
commonsenseqa_dataset = []
commonsenseqa_start_index = 0

if os.path.exists(COMMONSENSEQA_CHECKPOINT_FILE):
    try:
        with open(COMMONSENSEQA_CHECKPOINT_FILE, 'r') as f:
            commonsenseqa_dataset = json.load(f)
        commonsenseqa_start_index = len(commonsenseqa_dataset)
        print(f"📁 FOUND CHECKPOINT: {commonsenseqa_start_index} samples already completed")
        print(f"📊 Progress: {commonsenseqa_start_index}/{COMMONSENSEQA_TOTAL_SAMPLES} ({commonsenseqa_start_index/COMMONSENSEQA_TOTAL_SAMPLES*100:.1f}%)")
        
        if commonsenseqa_start_index > 0:
            print(f"✅ Resuming from sample #{commonsenseqa_start_index + 1}")
            print(f"🚀 Remaining: {COMMONSENSEQA_TOTAL_SAMPLES - commonsenseqa_start_index} samples")
        
    except Exception as e:
        print(f"❌ Error loading CommonSenseQA checkpoint: {e}")
        print("🔄 Starting fresh...")
        commonsenseqa_dataset = []
        commonsenseqa_start_index = 0
else:
    print("📄 No checkpoint found - starting from beginning")

print(f"\n🚀 COMMONSENSEQA COT GENERATION STATUS:")
print(f"   • Total target: {COMMONSENSEQA_TOTAL_SAMPLES}")
print(f"   • Already completed: {commonsenseqa_start_index}")
print(f"   • Remaining: {COMMONSENSEQA_TOTAL_SAMPLES - commonsenseqa_start_index}")
print(f"   • Checkpoint every: {COMMONSENSEQA_CHECKPOINT_EVERY} samples")
print(f"   • Recommended prompt: 'concept_guided'")
print(f"   • Rate limit: 4 seconds between requests")
print(f"   • Estimated time: ~{((COMMONSENSEQA_TOTAL_SAMPLES - commonsenseqa_start_index) * 4) // 60} minutes")
print("="*60)

🔄 PREPARING COMMONSENSEQA DATASET FOR COT GENERATION...
📊 CommonSenseQA Dataset prepared: 9741 problems
   Sample concepts: ['seafood restaurant', 'lying', 'distance', 'bench', 'choker', 'sun', 'skiing', 'fountain pen', 'run errands', 'watching tv']...
📄 No checkpoint found - starting from beginning

🚀 COMMONSENSEQA COT GENERATION STATUS:
   • Total target: 1000
   • Already completed: 0
   • Remaining: 1000
   • Checkpoint every: 25 samples
   • Recommended prompt: 'concept_guided'
   • Rate limit: 4 seconds between requests
   • Estimated time: ~66 minutes


In [17]:
# COMMONSENSEQA COT GENERATION LOOP
if commonsenseqa_start_index >= COMMONSENSEQA_TOTAL_SAMPLES:
    print("🎉 CommonSenseQA CoT generation already completed! All samples generated.")
    
    # Create final file if it doesn't exist
    final_file = f"cot_commonsenseqa_final_{len(commonsenseqa_dataset)}.json"
    if not os.path.exists(final_file):
        with open(final_file, "w") as f:
            json.dump(commonsenseqa_dataset, f, indent=2)
        print(f"🏁 Final dataset created: {final_file}")
    else:
        print(f"📁 Final file already exists: {final_file}")
else:
    remaining_samples = COMMONSENSEQA_TOTAL_SAMPLES - commonsenseqa_start_index
    print(f"🚀 Generating {remaining_samples} remaining CommonSenseQA CoT samples...")
    print(f"📍 Starting from sample #{commonsenseqa_start_index + 1}")
    print(f"💾 Checkpoints every {COMMONSENSEQA_CHECKPOINT_EVERY} samples")
    print("="*50)

    # Generate remaining samples
    for i, sample in enumerate(tqdm(commonsenseqa_questions[commonsenseqa_start_index:commonsenseqa_start_index + remaining_samples], 
                                  desc=f"Processing CommonSenseQA {commonsenseqa_start_index+1}-{COMMONSENSEQA_TOTAL_SAMPLES}", 
                                  unit="problem")):
        
        # Use concept_guided prompt for best reasoning with systematic approach
        entry = generate_cot_commonsenseqa(sample, "concept_guided")
        commonsenseqa_dataset.append(entry)
        
        # Checkpoint saving with Windows-compatible atomic write
        if len(commonsenseqa_dataset) % COMMONSENSEQA_CHECKPOINT_EVERY == 0:
            temp_file = f"temp_{COMMONSENSEQA_CHECKPOINT_FILE}"
            with open(temp_file, "w") as f:
                json.dump(commonsenseqa_dataset, f, indent=2)
            
            # Windows-compatible atomic write
            if os.path.exists(COMMONSENSEQA_CHECKPOINT_FILE):
                os.remove(COMMONSENSEQA_CHECKPOINT_FILE)
            os.rename(temp_file, COMMONSENSEQA_CHECKPOINT_FILE)
            print(f"💾 CommonSenseQA Checkpoint: {len(commonsenseqa_dataset)}/{COMMONSENSEQA_TOTAL_SAMPLES} samples saved")
        
        time.sleep(4)  # Rate limit delay

    # Final save
    with open(COMMONSENSEQA_CHECKPOINT_FILE, "w") as f:
        json.dump(commonsenseqa_dataset, f, indent=2)
    
    print(f"✅ CommonSenseQA Generation complete! {len(commonsenseqa_dataset)} samples saved to {COMMONSENSEQA_CHECKPOINT_FILE}")

    # Create final numbered file
    commonsenseqa_final_file = f"cot_commonsenseqa_final_{len(commonsenseqa_dataset)}.json"
    with open(commonsenseqa_final_file, "w") as f:
        json.dump(commonsenseqa_dataset, f, indent=2)
    print(f"🏁 Final CommonSenseQA dataset: {commonsenseqa_final_file}")
    
    # Show sample of generated data and accuracy
    if commonsenseqa_dataset:
        print(f"\n📋 SAMPLE GENERATED COMMONSENSEQA COT:")
        sample_entry = commonsenseqa_dataset[0]
        print(f"   ID: {sample_entry['id']}")
        print(f"   Concept: {sample_entry['question_concept']}")
        print(f"   Question: {sample_entry['question'][:100]}...")
        print(f"   Generated Answer: {sample_entry['ans']}")
        print(f"   Gold Answer: {sample_entry['gold']}")
        print(f"   CoT Length: {len(sample_entry['cot'])} characters")
        
        # Check accuracy on all samples
        correct = sum(1 for entry in commonsenseqa_dataset if entry['ans'] == entry['gold'])
        accuracy = correct / len(commonsenseqa_dataset) * 100
        print(f"   📊 Final Accuracy: {accuracy:.1f}% ({correct}/{len(commonsenseqa_dataset)})")

print("="*60)

🚀 Generating 1000 remaining CommonSenseQA CoT samples...
📍 Starting from sample #1
💾 Checkpoints every 25 samples


Processing CommonSenseQA 1-1000:   2%|▏         | 24/1000 [03:08<1:58:40,  7.30s/problem]

💾 CommonSenseQA Checkpoint: 25/1000 samples saved


Processing CommonSenseQA 1-1000:   5%|▍         | 49/1000 [06:22<1:49:43,  6.92s/problem]

💾 CommonSenseQA Checkpoint: 50/1000 samples saved


...

💾 CommonSenseQA Checkpoint: 975/1000 samples saved


Processing CommonSenseQA 1-1000: 100%|█████████▉| 999/1000 [2:03:23<00:07,  7.19s/problem]

💾 CommonSenseQA Checkpoint: 1000/1000 samples saved


Processing CommonSenseQA 1-1000: 100%|██████████| 1000/1000 [2:03:33<00:00,  7.41s/problem]

✅ CommonSenseQA Generation complete! 1000 samples saved to cot_commonsenseqa_checkpoint.json
🏁 Final CommonSenseQA dataset: cot_commonsenseqa_final_1000.json

📋 SAMPLE GENERATED COMMONSENSEQA COT:
   ID: 075e483d21c29a511267ef62bedc0461
   Concept: punishing
   Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the scho...
   Generated Answer: E
   Gold Answer: A
   CoT Length: 3237 characters
   📊 Final Accuracy: 64.7% (647/1000)


In [18]:
# ADD 'is_correct' COLUMN TO COMMONSENSEQA DATASET
print("🔧 ADDING 'is_correct' COLUMN TO COMMONSENSEQA DATASET")
print("="*60)

import json
import os

# File paths
input_file = "cot_commonsenseqa_final_1000.json"
output_file = "cot_commonsenseqa_final_1000_with_correct.json"

if os.path.exists(input_file):
    # Load the dataset
    with open(input_file, 'r') as f:
        commonsenseqa_data = json.load(f)
    
    print(f"📊 Loaded {len(commonsenseqa_data)} samples from {input_file}")
    
    # Add 'is_correct' column
    correct_count = 0
    for entry in commonsenseqa_data:
        # Compare generated answer with gold answer
        is_correct = entry['ans'] == entry['gold']
        entry['is_correct'] = is_correct
        
        if is_correct:
            correct_count += 1
    
    # Save the updated dataset
    with open(output_file, 'w') as f:
        json.dump(commonsenseqa_data, f, indent=2)
    
    # Calculate and display statistics
    total_samples = len(commonsenseqa_data)
    accuracy = (correct_count / total_samples) * 100
    
    print(f"✅ Successfully added 'is_correct' column!")
    print(f"📈 Dataset Statistics:")
    print(f"   • Total samples: {total_samples}")
    print(f"   • Correct answers: {correct_count}")
    print(f"   • Incorrect answers: {total_samples - correct_count}")
    print(f"   • Accuracy: {accuracy:.2f}%")
    print(f"📁 Updated dataset saved as: {output_file}")
    
    # Show sample entries with the new column
    print(f"\n📋 SAMPLE ENTRIES WITH 'is_correct' COLUMN:")
    for i, entry in enumerate(commonsenseqa_data[:3]):
        print(f"\nSample {i+1}:")
        print(f"   ID: {entry['id']}")
        print(f"   Concept: {entry['question_concept']}")
        print(f"   Question: {entry['question'][:80]}...")
        print(f"   Generated Answer: {entry['ans']}")
        print(f"   Gold Answer: {entry['gold']}")
        print(f"   Is Correct: {entry['is_correct']}")
    
    # Also update the original file (backup approach)
    backup_file = "cot_commonsenseqa_final_1000_backup.json"
    if not os.path.exists(backup_file):
        # Create backup of original
        with open(input_file, 'r') as f:
            original_data = json.load(f)
        with open(backup_file, 'w') as f:
            json.dump(original_data, f, indent=2)
        print(f"💾 Backup created: {backup_file}")
    
    # Overwrite original file with updated data
    with open(input_file, 'w') as f:
        json.dump(commonsenseqa_data, f, indent=2)
    print(f"🔄 Original file updated: {input_file}")
    
    # Show answer distribution analysis
    print(f"\n📊 ANSWER DISTRIBUTION ANALYSIS:")
    answer_dist = {}
    for entry in commonsenseqa_data:
        ans = entry['ans']
        answer_dist[ans] = answer_dist.get(ans, 0) + 1
    print(f"   Generated answers: {answer_dist}")
    
    # Show accuracy by concept (sample analysis)
    concept_accuracy = {}
    for entry in commonsenseqa_data[:100]:  # Sample first 100 for concept analysis
        concept = entry['question_concept']
        if concept not in concept_accuracy:
            concept_accuracy[concept] = {'correct': 0, 'total': 0}
        concept_accuracy[concept]['total'] += 1
        if entry['is_correct']:
            concept_accuracy[concept]['correct'] += 1
    
    print(f"\n🎯 CONCEPT ACCURACY ANALYSIS (First 100 samples):")
    for concept, stats in list(concept_accuracy.items())[:5]:
        acc = (stats['correct'] / stats['total']) * 100
        print(f"   {concept}: {acc:.1f}% ({stats['correct']}/{stats['total']})")
    
else:
    print(f"❌ File not found: {input_file}")
    print("   Make sure the CommonSenseQA final dataset file exists in the current directory")

print("="*60)

🔧 ADDING 'is_correct' COLUMN TO COMMONSENSEQA DATASET
📊 Loaded 1000 samples from cot_commonsenseqa_final_1000.json
✅ Successfully added 'is_correct' column!
📈 Dataset Statistics:
   • Total samples: 1000
   • Correct answers: 647
   • Incorrect answers: 353
   • Accuracy: 64.70%
📁 Updated dataset saved as: cot_commonsenseqa_final_1000_with_correct.json

📋 SAMPLE ENTRIES WITH 'is_correct' COLUMN:

Sample 1:
   ID: 075e483d21c29a511267ef62bedc0461
   Concept: punishing
   Question: The sanctions against the school were a punishing blow, and they seemed to what ...
   Generated Answer: E
   Gold Answer: A
   Is Correct: False

Sample 2:
   ID: 61fe6e879ff18686d7552425a36344c8
   Concept: people
   Question: Sammy wanted to go to where the people were.  Where might he go?...
   Generated Answer: B
   Gold Answer: B
   Is Correct: True

Sample 3:
   ID: 4c1cb0e95b99f72d55c068ba0255c54d
   Concept: choker
   Question: To locate a choker not located in a jewelry box or boutique where would yo